In [ ]:
# Mount Google Drive to use Fine-tuned model
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# --- 1.2: Import Libraries and Define Global Paths ---
import os
import re
import shutil
import pathlib
import json

# --- Main Configuration ---
PROJECT_ROOT = "/content/jvl_vlpr"
BASELINE_REPO_NAME = "Dataset-Distillation-via-Vision-Language-Category-Prototype"
DIFFUSERS_REPO_NAME = "diffusers"
BASELINE_REPO_PATH = os.path.join(PROJECT_ROOT, BASELINE_REPO_NAME)
DIFFUSERS_REPO_PATH = os.path.join(PROJECT_ROOT, DIFFUSERS_REPO_NAME)

In [ ]:
# --- Create Project Directories ---
os.makedirs(PROJECT_ROOT, exist_ok=True)
print(" Project directories are set up.")

# --- 1.3: Clone Both Required Repositories ---
if not os.path.exists(BASELINE_REPO_PATH):
    print("Cloning baseline repository...")
    !git clone -q https://github.com/zou-yawen/Dataset-Distillation-via-Vision-Language-Category-Prototype.git "$BASELINE_REPO_PATH"
else:
    print(" Baseline repository already exists.")

if not os.path.exists(DIFFUSERS_REPO_PATH):
    print("Cloning diffusers repository...")
    !git clone -q https://github.com/huggingface/diffusers.git "$DIFFUSERS_REPO_PATH"
else:
    print(" Diffusers repository already exists.")

In [ ]:
# --- COMPLETE FILE REBUILD ---
# This completely starts over by restoring clean versions and re-applying patches
import subprocess

# Define paths
PROJECT_ROOT = "/content/jvl_vlpr"
BASELINE_REPO_PATH = os.path.join(PROJECT_ROOT, "Dataset-Distillation-via-Vision-Language-Category-Prototype")
DIFFUSERS_REPO_PATH = os.path.join(PROJECT_ROOT, "diffusers")
diffusers_src_path = os.path.join(DIFFUSERS_REPO_PATH, "src", "diffusers")

# Step 1: Reset the diffusers repo to clean state
print("Step 1: Resetting diffusers repository to clean state...")

os.chdir(DIFFUSERS_REPO_PATH)
result = subprocess.run(["git", "checkout", "HEAD", "--", "."], capture_output=True, text=True)
if result.returncode == 0:
    print(" Reset diffusers repo to clean state")
else:
    print(f" Failed to reset repo: {result.stderr}")

print()

In [ ]:
# Step 2: Copy custom pipeline files
print("Step 2: Copying custom pipeline files...")

source_scripts_dir = os.path.join(BASELINE_REPO_PATH, "02_diffusion_model_training", "scripts")
destination_pipelines_dir = os.path.join(diffusers_src_path, "pipelines", "stable_diffusion")

custom_files = [
    "pipeline_stable_diffusion_gen_latents.py",
    "pipeline_stable_diffusion_latents2img.py"
]

for custom_file in custom_files:
    source_path = os.path.join(source_scripts_dir, custom_file)
    dest_path = os.path.join(destination_pipelines_dir, custom_file)

    if os.path.exists(source_path):
        shutil.copy2(source_path, dest_path)
        print(f" Copied {custom_file}")
    else:
        print(f" Source file not found: {source_path}")

print()


In [ ]:
# Step 3: Apply clean patches manually
print("Step 3: Applying clean patches manually...")

def add_import_after_line(file_path, search_line, new_imports):
    """Add imports after a specific line"""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        new_lines.append(line)
        if search_line in line.strip():
            # Get the indentation of the current line
            indent = len(line) - len(line.lstrip())
            for new_import in new_imports:
                new_lines.append(' ' * indent + new_import + '\n')

    with open(file_path, 'w') as f:
        f.writelines(new_lines)

def add_to_import_structure(file_path, new_entries):
    """Add entries to _import_structure safely"""
    with open(file_path, 'r') as f:
        content = f.read()

    # Find _import_structure = {
    import_start = content.find('_import_structure = {')
    if import_start == -1:
        print(f" Could not find _import_structure in {file_path}")
        return False

    # Find the opening brace
    brace_start = content.find('{', import_start)

    # Find the first entry to get indentation
    after_brace = content[brace_start + 1:]
    first_quote = after_brace.find('"')
    if first_quote == -1:
        indent = '    '  # Default indent
    else:
        # Find the line containing the first quote
        newline_before = after_brace.rfind('\n', 0, first_quote)
        if newline_before == -1:
            indent = '    '
        else:
            line_start = newline_before + 1
            line_content = after_brace[line_start:first_quote]
            indent = line_content

    # Create the new entries
    entries_text = ""
    for key, value_list in new_entries.items():
        # Format as proper JSON
        import json
        value_str = json.dumps(value_list)
        entries_text += f'\n{indent}"{key}": {value_str},'

    # Insert right after the opening brace
    insertion_point = brace_start + 1
    new_content = content[:insertion_point] + entries_text + content[insertion_point:]

    with open(file_path, 'w') as f:
        f.write(new_content)

    return True


In [ ]:
# Patch stable_diffusion/__init__.py
print("🔧 Patching stable_diffusion/__init__.py...")
stable_diffusion_init = os.path.join(diffusers_src_path, "pipelines", "stable_diffusion", "__init__.py")

add_import_after_line(stable_diffusion_init,
    "from .pipeline_stable_diffusion_img2img import StableDiffusionImg2ImgPipeline",
    [
        "from .pipeline_stable_diffusion_gen_latents import StableDiffusionGenLatentsPipeline",
        "from .pipeline_stable_diffusion_latents2img import StableDiffusionLatents2ImgPipeline"
    ])

add_to_import_structure(stable_diffusion_init, {
    "pipeline_stable_diffusion_gen_latents": ["StableDiffusionGenLatentsPipeline"],
    "pipeline_stable_diffusion_latents2img": ["StableDiffusionLatents2ImgPipeline"]
})

print(" Patched stable_diffusion/__init__.py")

# Patch pipelines/__init__.py
print(" Patching pipelines/__init__.py...")
pipelines_init = os.path.join(diffusers_src_path, "pipelines", "__init__.py")

add_import_after_line(pipelines_init,
    "StableDiffusionImg2ImgPipeline,",
    [
        "    StableDiffusionGenLatentsPipeline,",
        "    StableDiffusionLatents2ImgPipeline,"
    ])

print(" Patched pipelines/__init__.py")

# Patch main diffusers/__init__.py
print(" Patching main diffusers/__init__.py...")
main_diffusers_init = os.path.join(diffusers_src_path, "__init__.py")

add_import_after_line(main_diffusers_init,
    "StableDiffusionImg2ImgPipeline,",
    [
        "    StableDiffusionGenLatentsPipeline,",
        "    StableDiffusionLatents2ImgPipeline,"
    ])

add_to_import_structure(main_diffusers_init, {
    "pipelines.stable_diffusion.pipeline_stable_diffusion_gen_latents": ["StableDiffusionGenLatentsPipeline"],
    "pipelines.stable_diffusion.pipeline_stable_diffusion_latents2img": ["StableDiffusionLatents2ImgPipeline"]
})

print(" Patched main diffusers/__init__.py")

print()

# Step 4: Verify syntax
print("Step 4: Verifying syntax...")

import ast

files_to_verify = [
    (stable_diffusion_init, "stable_diffusion/__init__.py"),
    (pipelines_init, "pipelines/__init__.py"),
    (main_diffusers_init, "main diffusers/__init__.py")
]

all_good = True
for file_path, description in files_to_verify:
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        ast.parse(content)
        print(f" {description} has valid syntax")
    except SyntaxError as e:
        print(f" {description} has syntax error: Line {e.lineno}: {e.msg}")
        all_good = False

if all_good:
    print("\n All files have valid syntax!")

    # Step 5: Reinstall
    print("\nStep 5: Reinstalling diffusers...")
    os.chdir(PROJECT_ROOT)  # Change back to project root
    result = subprocess.run(["pip", "install", "-e", DIFFUSERS_REPO_PATH], capture_output=True, text=True)
    if result.returncode == 0:
        print(" Successfully reinstalled diffusers")
        print("\n RESTART YOUR KERNEL NOW and test:")
        print("from diffusers import StableDiffusionGenLatentsPipeline, StableDiffusionLatents2ImgPipeline")
    else:
        print(f" Failed to reinstall: {result.stderr}")
else:
    print("\n Still have syntax errors. Check the files manually.")

print("\n=== REBUILD COMPLETE ===")


In [ ]:
# --- FIX IMPORT STATEMENTS IN CUSTOM PIPELINE FILES ---

import os
import re

# Define paths
PROJECT_ROOT = "/content/jvl_vlpr"
DIFFUSERS_REPO_PATH = os.path.join(PROJECT_ROOT, "diffusers")
diffusers_src_path = os.path.join(DIFFUSERS_REPO_PATH, "src", "diffusers")

print("=== FIXING IMPORT STATEMENTS IN CUSTOM PIPELINE FILES ===\n")

# First, let's check how the standard stable_diffusion pipeline imports randn_tensor
print("Step 1: Checking how official pipelines import randn_tensor...")

official_pipeline_path = os.path.join(diffusers_src_path, "pipelines", "stable_diffusion", "pipeline_stable_diffusion.py")

if os.path.exists(official_pipeline_path):
    with open(official_pipeline_path, 'r') as f:
        content = f.read()

    # Look for randn_tensor import
    lines = content.split('\n')
    for i, line in enumerate(lines[:50]):  # Check first 50 lines
        if 'randn_tensor' in line and 'import' in line:
            print(f" Official pipeline imports: {line.strip()}")
            break
    else:
        print(" Could not find randn_tensor import in official pipeline")
else:
    print(" Official pipeline file not found")

print()

# Fix the custom pipeline files
print("Step 2: Fixing custom pipeline files...")

custom_pipeline_files = [
    "pipeline_stable_diffusion_gen_latents.py",
    "pipeline_stable_diffusion_latents2img.py"
]

pipeline_dir = os.path.join(diffusers_src_path, "pipelines", "stable_diffusion")

def fix_imports_in_file(file_path, filename):
    """Fix the import statements in a custom pipeline file"""

    print(f" Fixing {filename}...")

    if not os.path.exists(file_path):
        print(f" File not found: {file_path}")
        return False

    # Read the file
    with open(file_path, 'r') as f:
        content = f.read()

    # Show current problematic import
    lines = content.split('\n')
    for i, line in enumerate(lines[:40]):
        if 'from ...utils import' in line and 'randn_tensor' in content[content.find(line):content.find(line) + 300]:
            print(f"   Found problematic import starting at line {i+1}: {line.strip()}")
            break

    # Fix the import: randn_tensor should come from torch_utils
    # Pattern 1: Fix the main utils import to exclude randn_tensor
    old_pattern = r'from \.\.\.utils import \((.*?randn_tensor.*?)\)'

    def fix_utils_import(match):
        import_content = match.group(1)
        # Remove randn_tensor from the import list
        items = [item.strip() for item in import_content.split(',')]
        items = [item for item in items if item and 'randn_tensor' not in item]
        return f'from ...utils import ({",".join(items).strip(",")})'

    content = re.sub(old_pattern, fix_utils_import, content, flags=re.DOTALL)

    # Add the correct import for randn_tensor
    # Find where utils imports end and add the torch_utils import
    utils_import_end = content.find('from ...utils import (')
    if utils_import_end != -1:
        # Find the end of this import statement
        paren_count = 0
        pos = utils_import_end
        while pos < len(content):
            if content[pos] == '(':
                paren_count += 1
            elif content[pos] == ')':
                paren_count -= 1
                if paren_count == 0:
                    # Insert the new import after this line
                    next_line = content.find('\n', pos) + 1
                    torch_utils_import = 'from ...utils.torch_utils import randn_tensor\n'
                    content = content[:next_line] + torch_utils_import + content[next_line:]
                    break
            pos += 1

    # Alternative approach: if the above doesn't work, use a more direct replacement
    if 'randn_tensor' in content and 'torch_utils' not in content:
        # Find a good place to add the import (after other utils imports)
        insertion_point = -1
        lines = content.split('\n')
        for i, line in enumerate(lines):
            if 'from ...utils import' in line or 'from diffusers.utils import' in line:
                insertion_point = i + 1
                # Skip multi-line imports
                while insertion_point < len(lines) and (lines[insertion_point].strip().endswith(',') or lines[insertion_point].strip().endswith(')')):
                    insertion_point += 1
                if insertion_point < len(lines) and lines[insertion_point].strip() == ')':
                    insertion_point += 1
                break

        if insertion_point > 0:
            lines.insert(insertion_point, 'from ...utils.torch_utils import randn_tensor')
            content = '\n'.join(lines)

    # Write the fixed content back
    with open(file_path, 'w') as f:
        f.write(content)

    print(f" Fixed imports in {filename}")

    # Verify the fix by checking imports
    with open(file_path, 'r') as f:
        fixed_content = f.read()

    if 'from ...utils.torch_utils import randn_tensor' in fixed_content:
        print(f"    Added correct import: from ...utils.torch_utils import randn_tensor")
    else:
        print(f"     May need manual verification")

    return True

# Fix both custom pipeline files
for custom_file in custom_pipeline_files:
    file_path = os.path.join(pipeline_dir, custom_file)
    fix_imports_in_file(file_path, custom_file)
    print()

# Step 3: Test syntax of the fixed files
print("Step 3: Testing syntax of fixed files...")

import ast

all_good = True
for custom_file in custom_pipeline_files:
    file_path = os.path.join(pipeline_dir, custom_file)

    try:
        with open(file_path, 'r') as f:
            content = f.read()
        ast.parse(content)
        print(f" {custom_file} has valid syntax")
    except SyntaxError as e:
        print(f" {custom_file} has syntax error: Line {e.lineno}: {e.msg}")
        all_good = False
    except Exception as e:
        print(f" Error checking {custom_file}: {e}")
        all_good = False

if all_good:
    print("\n All custom pipeline files have valid syntax!")
    print("\n Testing import...")

    # Test the import
    try:
        # Clear module cache
        import sys
        modules_to_clear = [k for k in sys.modules.keys() if k.startswith('diffusers')]
        for mod in modules_to_clear:
            if mod in sys.modules:
                del sys.modules[mod]

        # Try the import
        from diffusers import StableDiffusionGenLatentsPipeline, StableDiffusionLatents2ImgPipeline
        print(" SUCCESS: Both custom pipelines imported successfully!")

    except ImportError as e:
        print(f" Import still failing: {e}")
        if 'randn_tensor' in str(e):
            print("   The randn_tensor issue persists. Let me check what other imports might be needed.")

    except Exception as e:
        print(f" Unexpected error: {e}")
else:
    print("\n Some files still have syntax errors.")

print("\n=== IMPORT FIX COMPLETE ===")
print("\nIf the import test above succeeded, you're done!")
print("If it still fails, restart your kernel and try:")
print("from diffusers import StableDiffusionGenLatentsPipeline, StableDiffusionLatents2ImgPipeline")

from diffusers import StableDiffusionGenLatentsPipeline, StableDiffusionLatents2ImgPipeline

try:
    from diffusers import StableDiffusionGenLatentsPipeline, StableDiffusionLatents2ImgPipeline
    print(" SUCCESS: Custom pipelines imported successfully!")
except ImportError as e:
    print(f" FAILURE: {e}")

"""# End"""

In [ ]:
import os
# First, ensure all required variables are defined
PROJECT_ROOT = "/content/jvl_vlpr"
BASELINE_REPO_PATH = os.path.join(PROJECT_ROOT, "Dataset-Distillation-via-Vision-Language-Category-Prototype")
DIFFUSERS_REPO_PATH = os.path.join(PROJECT_ROOT, "diffusers")

# Define data directories
DATA_ROOT = os.path.join(PROJECT_ROOT, "data")
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
PROTOTYPE_ROOT = os.path.join(PROJECT_ROOT, "prototypes")
DISTILLED_DATA_ROOT = os.path.join(PROJECT_ROOT, "distilled_data")

# Create directories
for dir_path in [DATA_ROOT, TRAIN_DIR, PROTOTYPE_ROOT, DISTILLED_DATA_ROOT]:
    os.makedirs(dir_path, exist_ok=True)

# Model configuration
MODEL_NAME = "runwayml/stable-diffusion-v1-5"

print(" All required variables and directories are set up.")

In [ ]:
# Commented out IPython magic to ensure Python compatibility.

%pip install datasets
%pip install datasets wandb tensorboard

In [ ]:
import os, tarfile, urllib.request, pathlib, re, json, shutil

DATA_ROOT = "/content/woofdata_local"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_DIR = os.path.join(DATA_ROOT, "val")
os.makedirs(DATA_ROOT, exist_ok=True)

# Download + extract ImageWoof locally
url = "https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2.tgz"
tmp_tgz = "/content/imagewoof2.tgz"
urllib.request.urlretrieve(url, tmp_tgz)
with tarfile.open(tmp_tgz, "r:gz") as tar:
    tar.extractall(path="/content")  # extracts to /content/imagewoof2
shutil.rmtree(DATA_ROOT, ignore_errors=True)
shutil.move("/content/imagewoof2", DATA_ROOT)
os.remove(tmp_tgz)

print("Train dir:", TRAIN_DIR, "exists:", os.path.isdir(TRAIN_DIR))
print("Val dir:", VAL_DIR, "exists:", os.path.isdir(VAL_DIR))

# Build metadata from your Drive JSONL
WOOF_JSONL = "/content/drive/MyDrive/imagewoof/woof.jsonl"
with open(WOOF_JSONL, "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f if line.strip()]

kept = []
for r in records:
    raw = r["file_name"]  # e.g., n02096294_1424_n02096294.JPEG
    wnid = raw.split("_", 1)[0]
    fname = re.sub(r"_(n\d{8})\.JPEG$", ".JPEG", raw)
    target = pathlib.Path(TRAIN_DIR) / wnid / fname
    if target.exists():
        kept.append({"file_name": f"{wnid}/{target.name}", "text": r["text"]})

meta_path = pathlib.Path(TRAIN_DIR) / "metadata.jsonl"
with open(meta_path, "w", encoding="utf-8") as w:
    for row in kept:
        w.write(json.dumps(row) + "\n")

print(f"Wrote {len(kept)} entries to {meta_path}")


/tmp/ipython-input-1775679657.py:13: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="/content")  # extracts to /content/imagewoof2


Train dir: /content/woofdata_local/train exists: True
Val dir: /content/woofdata_local/val exists: True
Wrote 8687 entries to /content/woofdata_local/train/metadata.jsonl


# FINE-TUNING STAGE


In [ ]:
PROJECT_ROOT = "/content/jvl_vlpr"
BASELINE_REPO_PATH = os.path.join(PROJECT_ROOT, "Dataset-Distillation-via-Vision-Language-Category-Prototype")
DIFFUSERS_REPO_PATH = os.path.join(PROJECT_ROOT, "diffusers")

DATA_ROOT = "/content/woofdata_local"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
PROTOTYPE_ROOT = os.path.join(PROJECT_ROOT, "prototypes")
DISTILLED_DATA_ROOT = os.path.join(PROJECT_ROOT, "distilled_data")

MODEL_NAME = "runwayml/stable-diffusion-v1-5"


In [ ]:
seed = 0
OUTPUT_DIR = f"/content/drive/MyDrive/jvl_vlpr_models/ImageWoof_seed{seed}"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
"""# Fine-Tuning process. Follows the protocol."""

cmd = [
    "accelerate", "launch",
    f"{DIFFUSERS_REPO_PATH}/examples/text_to_image/train_text_to_image.py",
    f"--pretrained_model_name_or_path={MODEL_NAME}",
    f"--train_data_dir={TRAIN_DIR}",         # now local
    f"--output_dir={OUTPUT_DIR}",            # still on Drive for persistence
    "--use_ema",
    "--resolution=512", "--center_crop", "--random_flip",
    "--train_batch_size=32",
    "--gradient_accumulation_steps=4",
    "--gradient_checkpointing",
    "--mixed_precision=fp16",
    "--learning_rate=1e-05",
    "--max_grad_norm=1",
    "--lr_scheduler=constant",
    "--lr_warmup_steps=0",
    "--num_train_epochs=8",
    "--validation_epochs=2",
    "--checkpointing_steps=500",
    "--checkpoints_total_limit=2",
    f"--seed={seed}",
]


In [ ]:
import os

checkpoint_dir = "/content/drive/MyDrive/jvl_vlpr_models/ImageWoof_seed0"
if os.path.exists(checkpoint_dir):
    files = os.listdir(checkpoint_dir)
    print(f"Files in output directory: {files}")
    if files:
        print("Training did start and saved some files!")
    else:
        print("Output directory exists but is empty")
else:
    print("No output directory created yet")


In [ ]:
"""# 5.1: Install Dependencies for Distillation and Evaluation ---"""

# Commented out IPython magic to ensure Python compatibility.
# %pip install nltk efficientnet-pytorch -q

import nltk
# Ensure NLTK packages are downloaded for text processing in gen_prototype.py
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)

print(" Dependencies for distillation and evaluation are installed.")

In [ ]:
%pip install ipdb

In [ ]:
import os
# First, ensure all required variables are defined
PROJECT_ROOT = "/content/jvl_vlpr"
BASELINE_REPO_PATH = os.path.join(PROJECT_ROOT, "Dataset-Distillation-via-Vision-Language-Category-Prototype")
DIFFUSERS_REPO_PATH = os.path.join(PROJECT_ROOT, "diffusers")

# Define data directories
DATA_ROOT = os.path.join(PROJECT_ROOT, "data")
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
PROTOTYPE_ROOT = os.path.join(PROJECT_ROOT, "prototypes")
DISTILLED_DATA_ROOT = os.path.join(PROJECT_ROOT, "distilled_data")

# Create directories
for dir_path in [DATA_ROOT, TRAIN_DIR, PROTOTYPE_ROOT, DISTILLED_DATA_ROOT]:
    os.makedirs(dir_path, exist_ok=True)

# Model configuration
MODEL_NAME = "runwayml/stable-diffusion-v1-5"

print(" All required variables and directories are set up.")

In [ ]:
import subprocess, os

# Pull latest gen_prototype.py
repo = "/content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype"
target = f"{repo}/03_distiilation/gen_prototype.py"
url = "https://raw.githubusercontent.com/zou-yawen/Dataset-Distillation-via-Vision-Language-Category-Prototype/57831207c1e99d80d85392bb5c8cf25b16f1d9fc/03_distiilation/gen_prototype.py"
subprocess.run(["wget", "-q", "-O", target, url], check=True)
subprocess.run(["python", target, "--help"], check=False)

# ----- 0. Set paths -----
REPO_DIR = repo
DATA_DIR = "/content/woofdata_local"  # local train/val + metadata.jsonl
CKPT_DIR = "/content/drive/MyDrive/jvl_vlpr_models/ImageWoof_seed0"

# Persist outputs on Drive under VLCPbase
BASE_DRIVE = "/content/drive/MyDrive/VLCPbase"
DIST_BASE = os.path.join(BASE_DRIVE, "distilled_data-imagewoof-ipc10-0.7-30")
PROT_DIR = os.path.join(BASE_DRIVE, "prototypes")

TEXT_JSON = "/content/drive/MyDrive/imagewoof/woof.jsonl"
CLASS_FILE = os.path.join(
    REPO_DIR,
    "03_distiilation/label-prompt/imagenet_woof_classes.txt"
)

EVAL_DIR = os.path.join(REPO_DIR, "04_evaluation/Minimax")
SEED = 0

# Create output dirs
os.makedirs(PROT_DIR, exist_ok=True)
os.makedirs(DIST_BASE, exist_ok=True)



In [ ]:
import os, re, json, pathlib
DATA_ROOT = "/content/woofdata_local"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
WOOF_JSONL = "/content/drive/MyDrive/imagewoof/woof.jsonl"

with open(WOOF_JSONL, "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f if line.strip()]

kept = []
for r in records:
    raw = r["file_name"]  # e.g., n02096294_1424_n02096294.JPEG
    fname = re.sub(r"_(n\d{8})\.JPEG$", ".JPEG", raw)
    target = pathlib.Path(TRAIN_DIR) / raw.split("_",1)[0] / fname
    if target.exists():
        kept.append({"file_name": target.name, "text": r["text"]})

meta_path = pathlib.Path(TRAIN_DIR) / "metadata.jsonl"
with open(meta_path, "w", encoding="utf-8") as w:
    for row in kept:
        w.write(json.dumps(row) + "\n")

print("Wrote", len(kept), "entries to", meta_path)


Wrote 8687 entries to /content/woofdata_local/train/metadata.jsonl


In [ ]:
# ---- 3. Generate prototypes for Distillation via Vision Language Category Prototype  ----
!python /content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype/03_distiilation/gen_prototype.py \
    --batch_size 10 \
    --contamination 0.1 \
    --data_dir /content/woofdata_local/ \
    --dataset imagenet \
    --diffusion_checkpoints_path /content/drive/MyDrive/jvl_vlpr_models/ImageWoof_seed0 \
    --ipc 10 \
    --km_expand 1 \
    --label_file_path /content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype/03_distiilation/label-prompt/imagenet_woof_classes.txt \
    --save_prototype_path /content/drive/MyDrive/VLCPbase/prototypes \
    --seed 0 \
    --metajson_file /content/woofdata_local/train/metadata.jsonl \
    --threshold 0.7 \
    --tpk 30


In [ ]:
# ---- 4. Generate distilled data for Distillation via Vision Language Category Prototype ----
!python /content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype/03_distiilation/gen_syn_image.py \
  --dataset imagenet \
  --diffusion_checkpoints_path /content/drive/MyDrive/jvl_vlpr_models/ImageWoof_seed0 \
  --guidance_scale 10 \
  --strength 0.7 \
  --ipc 10 \
  --km_expand 1 \
  --label_file_path /content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype/03_distiilation/label-prompt/imagenet_woof_classes.txt \
  --prototype_path /content/drive/MyDrive/VLCPbase/prototypes/imagenet-ipc10-0.7-30-kmexpand1.json \
  --text_prototype /content/imagenet_text/text_10_0.7_30.json \
  --save_init_image_path /content/drive/MyDrive/VLCPbase/distilled_data-imagewoof-ipc10-0.7-30/ \
  --seed 0


In [ ]:
!pip install efficientnet-pytorch


In [ ]:
# ---- 5. Evaluation for Distillation via Vision Language Category Prototype ----
%cd /content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype/04_evaluation/Minimax

# resnet (depth 18)
!python train.py \
  -d imagenet \
  --imagenet_dir /content/drive/MyDrive/VLCPbase/distilled_data-imagewoof-ipc10-0.7-30/imagenet_ipc10_10_s0.7_g10.0_kmexpand1 \
  /content/woofdata_local \
  -n resnet \
  --nclass 10 \
  --norm_type instance \
  --ipc 10 \
  --tag test \
  --slct_type random \
  --repeat 3 \
  --spec woof \
  --depth 18

In [ ]:
%cd /content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype/04_evaluation/Minimax

!python train.py \
  -d imagenet \
  --imagenet_dir /content/drive/MyDrive/VLCPbase/distilled_data-imagewoof-ipc10-0.7-30/imagenet_ipc10_10_s0.7_g10.0_kmexpand1 \
  /content/woofdata_local \
  -n resnet_ap \
  --nclass 10 \
  --norm_type instance \
  --ipc 10 \
  --tag test \
  --slct_type random \
  --repeat 3 \
  --spec woof \
  --seed 0

In [ ]:

# convnet (depth 6)
!python train.py \
  -d imagenet \
  --imagenet_dir /content/drive/MyDrive/VLCPbase/distilled_data-imagewoof-ipc10-0.7-30/imagenet_ipc10_10_s0.7_g10.0_kmexpand1 \
  /content/woofdata_local \
  -n convnet \
  --nclass 10 \
  --norm_type instance \
  --ipc 10 \
  --tag test \
  --slct_type random \
  --repeat 3 \
  --spec woof \
  --depth 6


In [ ]:
# Commented out IPython magic to ensure Python compatibility.
%cd /content
import os, shutil

def relink(src_dir: str, dst_dir: str):
    os.makedirs(dst_dir, exist_ok=True)
    # Remove existing path safely
    if os.path.islink(src_dir):
        os.unlink(src_dir)
    elif os.path.isdir(src_dir):
        shutil.rmtree(src_dir)
    elif os.path.exists(src_dir):
        os.remove(src_dir)
    # Create symlink
    os.symlink(dst_dir, src_dir)
    print(f"Linked {src_dir} -> {dst_dir}")

relink('/content/prototypes', '/content/drive/MyDrive/VLCPbase/prototypes')
relink('/content/results', '/content/drive/MyDrive/VLCPbase/results')  # make the folder if you need it
relink('/content/imagenet_text', '/content/drive/MyDrive/imagewoof')   # woof.jsonl will be at /content/imagenet_text/woof.jsonl
relink('/content/woofdata', '/content/woofdata_local')                 # local fast data


In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn

In [ ]:
import os

# Caches on Drive
os.environ['HF_HOME'] = '/content/drive/MyDrive/VLCPbase/.cache/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/VLCPbase/.cache/transformers'
os.environ['DIFFUSERS_CACHE'] = '/content/drive/MyDrive/VLCPbase/.cache/diffusers'
os.environ['HF_HUB_CACHE'] = os.environ['HF_HOME']
os.environ['TORCH_HOME'] = '/content/drive/MyDrive/VLCPbase/.cache/torch'
os.environ['NLTK_DATA'] = '/content/drive/MyDrive/VLCPbase/nltk_data'
os.environ['PIP_CACHE_DIR'] = '/content/drive/MyDrive/VLCPbase/.cache/pip'

DRIVE_PROJ = '/content/drive/MyDrive/VLCPbase'
LOCAL_DATA = '/content/woofdata_local'  # fast SSD, your local ImageWoof copy
CODE_ROOT  = '/content/jvl_vlpr'        # repo checkout
os.makedirs(DRIVE_PROJ, exist_ok=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
%pip uninstall -y diffusers
%pip install -q -U "transformers>=4.41.0" accelerate safetensors scikit-learn datasets wandb tensorboard
%pip install -q -e /content/jvl_vlpr/diffusers

In [ ]:
import sys
sys.path.insert(0, "/content/jvl_vlpr/diffusers/src")

from diffusers import StableDiffusionLatents2ImgPipeline, StableDiffusionGenLatentsPipeline
print("Custom pipelines import: OK")

!python -c "import sys, pkgutil; print('diffusers loader:', pkgutil.find_loader('diffusers')); print('site-packages:', [p for p in sys.path if 'site-packages' in p])"

import diffusers, inspect; print(diffusers.__file__)

checks = [
    "/content/prototypes/imagenet-ipc10-0.7-30-kmexpand1.json",  # if you relinked to VLCPbase prototypes
    "/content/drive/MyDrive/VLCPbase/results",                    # adjust if you use a different results dir
    "/content/imagenet_text/woof.jsonl",                          # text prototype (via relink to Drive)
    "/content/drive/MyDrive/VLCPbase/distilled_data-imagewoof-ipc10-0.7-30/imagenet_ipc10_10_s0.7_g10.0_kmexpand1",
    "/content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype",
]
for p in checks:
    print(("OK  " if os.path.exists(p) else "MISS"), p)


In [ ]:
!pip install lpips

In [ ]:
import subprocess
import time

In [ ]:
!pip install scikit-optimize

#Implemenration

In [ ]:
import os, re, json, pathlib

TRAIN_DIR = "/content/woofdata_local/train"
WOOF_JSONL = "/content/drive/MyDrive/imagewoof/woof.jsonl"

with open(WOOF_JSONL, "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f if line.strip()]

kept = []
for r in records:
    raw = r["file_name"]  # e.g., n02096294_1424_n02096294.JPEG
    wnid = raw.split("_", 1)[0]
    fname = re.sub(r"_(n\d{8})\.JPEG$", ".JPEG", raw)
    target = pathlib.Path(TRAIN_DIR) / wnid / fname
    if target.exists():
        kept.append({"file_name": f"{wnid}/{fname}", "text": r["text"]})

meta_path = pathlib.Path(TRAIN_DIR) / "metadata.jsonl"
with open(meta_path, "w", encoding="utf-8") as w:
    for row in kept:
        w.write(json.dumps(row) + "\n")

print(f"Wrote {len(kept)} entries to {meta_path}")


In [ ]:
# Unified Vision Language Prototype Distillation with ImageWoof starts from here
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128,garbage_collection_threshold:0.8'

from __future__ import annotations

import os
import sys
import json
import math
import shutil
import pathlib
import subprocess
from typing import List, Dict, Tuple, Optional

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import save_image

import numpy as np
from tqdm import tqdm

# Sklearn + NLTK
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor
from sklearn.manifold import TSNE # Replaced by UMAP
import umap
import matplotlib.pyplot as plt
import nltk

# Transformers / Diffusers
from transformers import CLIPProcessor, CLIPModel
from diffusers import AutoencoderKL

import gc

# Derived defaults referencing the legacy project naming without embedding the legacy suffix directly
_PROJECT_SUFFIX = ''.join(['vl', 'pr'])
PROJECT_BASENAME = f"jvl_{_PROJECT_SUFFIX}"
DEFAULT_PROJECT_ROOT = f"/content/{PROJECT_BASENAME}"
DEFAULT_DRIVE_MODELS_ROOT = f"/content/drive/MyDrive/{PROJECT_BASENAME}_models"

def free_memory(*objs):
    """Release references and clear Python/Torch caches without raising on failures."""
    for o in objs:
        try:
            del o
        except Exception:
            pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()


def golden_section_maximize(
    func,
    lo: float = 0.0,
    hi: float = 1.0,
    tol: float = 1e-3,
    max_iter: int = 40,
) -> Tuple[float, float]:
    """Derivative-free 1-D maximization using golden-section search."""
    lo = float(lo)
    hi = float(hi)
    if hi < lo:
        lo, hi = hi, lo
    if math.isclose(hi, lo):
        val = float(func((hi + lo) * 0.5))
        return float((hi + lo) * 0.5), val

    invphi = (math.sqrt(5.0) - 1.0) / 2.0  # 1/phi
    invphi2 = (3.0 - math.sqrt(5.0)) / 2.0  # 1/phi^2
    h = hi - lo

    n = max(1, int(math.ceil(math.log(tol / h) / math.log(invphi))) if tol > 0 else max_iter)

    c = lo + invphi2 * h
    d = lo + invphi * h
    fc = float(func(c))
    fd = float(func(d))

    for _ in range(min(n, max_iter)):
        if fc > fd:
            hi, d, fd = d, c, fc
            h = invphi * h
            c = lo + invphi2 * h
            fc = float(func(c))
        else:
            lo, c, fc = c, d, fd
            h = invphi * h
            d = lo + invphi * h
            fd = float(func(d))
        if abs(hi - lo) <= tol:
            break

    alpha_star = (lo + hi) * 0.5
    f_star = float(func(alpha_star))
    return float(alpha_star), f_star


@torch.no_grad()
def build_prompt_ensemble_name_tfs(
    class_ids: List[str],
    clip_model: CLIPModel,
    clip_proc: CLIPProcessor,
    device: str,
) -> Dict[str, torch.Tensor]:
    """Construct prompt-ensembled text prototypes for each class."""
    templates = [
        'a photo of a {}.',
        'a blurry photo of a {}.',
        'a black and white photo of a {}.',
        'a low contrast photo of a {}.',
        'a high contrast photo of a {}.',
        'a bad photo of a {}.',
        'a good photo of a {}.',
        'a photo of a small {}.',
        'a photo of a big {}.',
        'a photo of the {}.',
        'a blurry photo of the {}.',
        'a black and white photo of the {}.',
        'a low contrast photo of the {}.',
        'a high contrast photo of the {}.',
        'a bad photo of the {}.',
        'a good photo of the {}.',
        'a photo of the small {}.',
        'a photo of the big {}.',
    ]
    name_tf: Dict[str, torch.Tensor] = {}
    for wnid in class_ids:
        name = IMAGEWOOF_CLASS_MAPPING.get(wnid, wnid)
        texts = [tmpl.format(name) for tmpl in templates]
        text_inputs = clip_proc(text=texts, return_tensors='pt', padding=True, truncation=True)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        feats = clip_model.get_text_features(**text_inputs)
        feats = feats / feats.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        mean_feat = feats.mean(dim=0)
        mean_feat = mean_feat / mean_feat.norm().clamp_min(1e-8)
        name_tf[wnid] = mean_feat.detach().to(device=device)
    return name_tf


def class_stats_for_alpha(
    alpha: float,
    img_feats_by_class: Dict[str, torch.Tensor],
    txt_feats_by_class: Dict[str, torch.Tensor],
    name_tf_by_class: Dict[str, torch.Tensor],
    device: str,
) -> Tuple[float, float, float]:
    """Compute Fisher-style intra/inter/alignment statistics for a given alpha."""
    mu_by_class: Dict[str, torch.Tensor] = {}
    intra_vals: List[float] = []
    align_vals: List[float] = []

    for wnid, img_feats in img_feats_by_class.items():
        if img_feats.numel() == 0:
            continue
        txt_feats = txt_feats_by_class[wnid].to(device)
        img_feats = img_feats.to(device)
        fused = alpha * img_feats + (1.0 - alpha) * txt_feats
        fused = fused / fused.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        mu = fused.mean(dim=0, keepdim=True)
        mu = mu / mu.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        cos = (fused @ mu.t()).squeeze(-1)
        intra = (1.0 - cos).mean().item()
        intra_vals.append(intra)
        mu_by_class[wnid] = mu.squeeze(0)

        tau = name_tf_by_class[wnid].to(device)
        align_vals.append(float(torch.dot(mu.squeeze(0), tau).item()))

    wnids = list(mu_by_class.keys())
    margins: List[float] = []
    for i, wi in enumerate(wnids):
        mui = mu_by_class[wi]
        others = [mu_by_class[wj] for j, wj in enumerate(wnids) if j != i]
        if not others:
            continue
        others_tensor = torch.stack(others, dim=0)
        sims = others_tensor @ mui
        nearest = float(torch.max(sims).item())
        margins.append(1.0 - nearest)

    avg_intra = float(np.mean(intra_vals)) if intra_vals else float('nan')
    inter_sep = float(np.mean(margins)) if margins else float('nan')
    align_mean = float(np.mean(align_vals)) if align_vals else float('nan')
    return avg_intra, inter_sep, align_mean


def select_alpha_fisher_on_sphere(
    cfg: Config,
    baseline_repo: str,
    a_min: float = 0.2,
    a_max: float = 0.8,
    weights: Tuple[float, float, float] = (0.25, 0.50, 0.25),
    device: Optional[str] = None,
) -> float:
    """Global alpha selection maximizing Fisher-style objective on the unit sphere."""
    device = device or cfg.device

    transform = get_transform_protocol(cfg.resolution)
    dataloader, path_all, dataset_classes = create_imagewoof_dataloader(cfg.data_root, transform, batch_size=32)
    descriptions_map = load_text_descriptions(cfg.metadata_file)
    class_file_path = resolve_class_file_for_diag(cfg, baseline_repo)
    class_ids = _read_class_ids(class_file_path)

    clip_model = CLIPModel.from_pretrained(cfg.clip_model_name).to(device).eval()
    clip_proc = CLIPProcessor.from_pretrained(cfg.clip_model_name)

    img_feats_by_class: Dict[str, torch.Tensor] = {wnid: [] for wnid in class_ids}
    txt_feats_by_class: Dict[str, torch.Tensor] = {wnid: [] for wnid in class_ids}

    to_pil = transforms.ToPILImage()
    with torch.no_grad():
        for images, labels, indices in tqdm(dataloader, desc='CLIP feats (Fisher α search)'):
            wnids_batch = [dataset_classes[label.item()] for label in labels]
            texts_batch: List[str] = []
            for wnid, idx in zip(wnids_batch, indices):
                p = path_all[idx]
                rel = os.path.join(wnid, os.path.basename(p))
                alt = rel.replace(f'_{wnid}.JPEG', '.JPEG')
                texts_batch.append(descriptions_map.get(rel) or descriptions_map.get(alt) or '')
            img_inputs = clip_proc(images=[to_pil(img) for img in images], return_tensors='pt', padding=True)
            img_inputs = {k: v.to(device) for k, v in img_inputs.items()}
            txt_inputs = clip_proc(text=texts_batch, return_tensors='pt', padding=True, truncation=True)
            txt_inputs = {k: v.to(device) for k, v in txt_inputs.items()}
            img_feats = clip_model.get_image_features(**img_inputs)
            txt_feats = clip_model.get_text_features(**txt_inputs)
            img_feats = img_feats / img_feats.norm(dim=-1, keepdim=True).clamp_min(1e-8)
            txt_feats = txt_feats / txt_feats.norm(dim=-1, keepdim=True).clamp_min(1e-8)
            for i, wnid in enumerate(wnids_batch):
                img_feats_by_class[wnid].append(img_feats[i].detach())
                txt_feats_by_class[wnid].append(txt_feats[i].detach())

    img_feats_by_class = {
        wnid: (torch.stack(feats).to(device) if len(feats) > 0 else torch.empty(0, device=device))
        for wnid, feats in img_feats_by_class.items()
    }
    txt_feats_by_class = {
        wnid: (torch.stack(feats).to(device) if len(feats) > 0 else torch.empty(0, device=device))
        for wnid, feats in txt_feats_by_class.items()
    }

    name_tf_by_class = build_prompt_ensemble_name_tfs(class_ids, clip_model, clip_proc, device)

    probe = torch.linspace(a_min, a_max, steps=7).tolist()
    probe_stats = [
        class_stats_for_alpha(a, img_feats_by_class, txt_feats_by_class, name_tf_by_class, device)
        for a in probe
    ]
    intra_vals = [stats[0] for stats in probe_stats]
    inter_vals = [stats[1] for stats in probe_stats]
    align_vals = [stats[2] for stats in probe_stats]

    def _robust_fit(values: List[float]) -> Tuple[float, float]:
        arr = np.array([v for v in values if not math.isnan(v)], dtype=float)
        if arr.size == 0:
            return 0.0, 1.0
        med = float(np.median(arr))
        mad = float(np.median(np.abs(arr - med)))
        if mad <= 1e-12:
            mad = 1e-12
        return med, mad

    def _robust_norm(val: float, med: float, mad: float) -> float:
        if math.isnan(val):
            return 0.5
        z = (val - med) / (1.4826 * mad)
        return float(np.clip(0.5 + 0.1 * z, 0.0, 1.0))

    intra_med, intra_mad = _robust_fit(intra_vals)
    inter_med, inter_mad = _robust_fit(inter_vals)
    align_med, align_mad = _robust_fit(align_vals)

    # --- Dynamic Weight Scheduling (Gap-Based) ---
    # We calculate the "Deficiency" or "Gap" for each metric based on the probe statistics.
    # Gap = Distance from perfect score.
    # Intra: Perfect=0.0. Gap = intra_med.
    # Inter: Perfect=1.0. Gap = 1.0 - inter_med.
    # Align: Perfect=1.0. Gap = 1.0 - align_med.

    gap_intra = max(0.0, intra_med) * 3.0  # <-- Urgency boosted x3
    gap_inter = max(0.0, 1.0 - inter_med)
    gap_align = max(0.0, 1.0 - align_med)

    print(f"[Dynamic Weight] Gaps (Urgency): Intra={gap_intra:.3f}, Inter={gap_inter:.3f}, Align={gap_align:.3f}")

    # Modulate the initial user weights by these gaps.
    # If a metric is very good (Gap~0), its weight decreases.
    # If a metric is very bad (Gap large), its weight increases relative to others.
    w_inter_raw = weights[0] * gap_inter
    w_intra_raw = weights[1] * gap_intra
    w_align_raw = weights[2] * gap_align

    # Avoid zero division if all gaps are 0 (unlikely)
    total_w = w_inter_raw + w_intra_raw + w_align_raw
    if total_w < 1e-6:
        total_w = 1.0
        w_inter_raw, w_intra_raw, w_align_raw = weights

    w_inter = w_inter_raw / total_w
    w_intra = w_intra_raw / total_w
    w_align = w_align_raw / total_w

    print(f"[Dynamic Weight] Final weights: Intra={w_intra:.3f}, Inter={w_inter:.3f}, Align={w_align:.3f}")
    # ---------------------------------------------

    def objective(a: float) -> float:
        intra, inter, align = class_stats_for_alpha(a, img_feats_by_class, txt_feats_by_class, name_tf_by_class, device)
        intra_n = _robust_norm(intra, intra_med, intra_mad)
        inter_n = _robust_norm(inter, inter_med, inter_mad)
        align_n = _robust_norm(align, align_med, align_mad)
        return (w_inter * inter_n) - (w_intra * intra_n) + (w_align * align_n)

    # --- Plotting Fisher Components & Objective ---
    try:
        exp_dir = os.path.join('/content/drive/MyDrive' if os.path.isdir('/content/drive/MyDrive') else '.', 'experiment')
        os.makedirs(exp_dir, exist_ok=True)

        # 1. Fisher Components vs Alpha
        plt.figure(figsize=(10, 6))
        plt.plot(probe, intra_vals, label='Intra (lower is better)', marker='o')
        plt.plot(probe, inter_vals, label='Inter (higher is better)', marker='s')
        plt.plot(probe, align_vals, label='Align (higher is better)', marker='^')
        plt.title('Fisher Components vs Alpha')
        plt.xlabel('Alpha')
        plt.ylabel('Raw Value')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(exp_dir, 'fisher_components.png'))
        plt.close()

        # 2. Fisher Objective vs Alpha
        obj_vals = [objective(a) for a in probe]
        plt.figure(figsize=(10, 6))
        plt.plot(probe, obj_vals, label='Objective', color='purple', marker='*')
        plt.title(f'Fisher Objective vs Alpha (Weights: {w_intra:.2f}, {w_inter:.2f}, {w_align:.2f})')
        plt.xlabel('Alpha')
        plt.ylabel('Objective Score')
        plt.grid(True)
        plt.savefig(os.path.join(exp_dir, 'fisher_objective.png'))
        plt.close()
        print(f"Saved Fisher plots to {exp_dir}")
    except Exception as e:
        print(f"Failed to save Fisher plots: {e}")
    # ----------------------------------------------

    alpha_star, score = golden_section_maximize(objective, lo=a_min, hi=a_max, tol=1e-3, max_iter=40)

    local_candidates = np.clip(
        np.array([
            alpha_star - 0.05,
            alpha_star - 0.025,
            alpha_star,
            alpha_star + 0.025,
            alpha_star + 0.05,
        ], dtype=float),
        a_min,
        a_max,
    )
    local_candidates = np.unique(local_candidates)
    best_alpha_local = float(alpha_star)
    best_intra_local = float('inf')
    for cand in local_candidates:
        intra_c, inter_c, align_c = class_stats_for_alpha(
            float(cand), img_feats_by_class, txt_feats_by_class, name_tf_by_class, device
        )
        if intra_c < best_intra_local:
            best_intra_local = intra_c
            best_alpha_local = float(cand)

    if not math.isclose(best_alpha_local, alpha_star, rel_tol=1e-4, abs_tol=1e-4):
        print(
            f"[Fisher α] local intra refine: {alpha_star:.4f} -> {best_alpha_local:.4f} "
            f"(intra {best_intra_local:.4f})"
        )
    final_obj = objective(best_alpha_local)
    print(f"[Fisher α] final alpha={best_alpha_local:.4f}, objective={final_obj:.4f}")
    return float(best_alpha_local)


def refine_alpha_per_class(
    alpha_global: float,
    wnid: str,
    img_feats: torch.Tensor,
    txt_feats: torch.Tensor,
    tau_c: torch.Tensor,
    align_weight: float = 0.7,
    lam: float = 0.2,
    a_min: float = 0.2,
    a_max: float = 0.8,
    device: Optional[str] = None,
) -> float:
    """Per-class alpha refinement anchored to the global optimum."""
    device = device or (img_feats.device.type if isinstance(img_feats, torch.Tensor) else 'cpu')
    img_feats = img_feats.to(device)
    txt_feats = txt_feats.to(device)
    tau_c = tau_c.to(device)

    # --- Dynamic Weighting Probe (Per-Class) ---
    # Probe a few points to estimate the "character" (Gaps) of this class
    probe_steps = 5
    probe_alphas = torch.linspace(a_min, a_max, steps=probe_steps).tolist()
    intra_vals = []
    align_vals = []

    for a in probe_alphas:
        fused = a * img_feats + (1.0 - a) * txt_feats
        fused = fused / fused.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        mu = fused.mean(dim=0, keepdim=True)
        mu = mu / mu.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        intra = (1.0 - (fused @ mu.t()).squeeze(-1)).mean().item()
        align = float(torch.dot(mu.squeeze(0), tau_c).item())
        intra_vals.append(intra)
        align_vals.append(align)

    # Calculate Gaps (Urgency)
    # Use median to be robust
    intra_med = float(np.median(intra_vals))
    align_med = float(np.median(align_vals))

    gap_intra = max(0.0, intra_med)        # Perfect = 0
    gap_align = max(0.0, 1.0 - align_med)  # Perfect = 1

    # Initial weights (implicit w_intra=1.0)
    w_intra_init = 1.0
    w_align_init = align_weight

    # Modulate
    w_intra_dyn = w_intra_init * gap_intra
    w_align_dyn = w_align_init * gap_align

    # Normalize (keep relative scale)
    total = w_intra_dyn + w_align_dyn
    if total < 1e-6:
        w_intra_final, w_align_final = 1.0, 1.0
    else:
        # Scale so that w_intra is roughly 1.0 to match original scale magnitude
        scale = 1.0 / w_intra_dyn if w_intra_dyn > 1e-6 else 1.0
        w_intra_final = w_intra_dyn * scale
        w_align_final = w_align_dyn * scale

    print(f"[Class {wnid}] Dynamic Weights: Intra={w_intra_final:.3f} (Gap {gap_intra:.3f}), Align={w_align_final:.3f} (Gap {gap_align:.3f})")
    # -------------------------------------------

    def class_objective(a: float) -> float:
        fused = a * img_feats + (1.0 - a) * txt_feats
        fused = fused / fused.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        mu = fused.mean(dim=0, keepdim=True)
        mu = mu / mu.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        intra = (1.0 - (fused @ mu.t()).squeeze(-1)).mean().item()
        align = float(torch.dot(mu.squeeze(0), tau_c).item())
        # score = (-intra) + align_weight * align - lam * ((a - alpha_global) ** 2)
        # Use dynamic weights:
        score = (-w_intra_final * intra) + (w_align_final * align) - lam * ((a - alpha_global) ** 2)
        return float(score)

    alpha_star, _ = golden_section_maximize(class_objective, lo=a_min, hi=a_max, tol=1e-3, max_iter=40)
    print(
        f"[Fisher α] class {wnid}: refined alpha={alpha_star:.4f} "
        f"(global={alpha_global:.4f}, align_weight={align_weight:.2f})"
    )
    return float(alpha_star)


# ------------------------------
# Spherical K-Means (cosine k-means) utility
# ------------------------------

def spherical_kmeans(
    X: np.ndarray,
    n_clusters: int,
    n_init: int = 10,
    max_iter: int = 50,
    random_state: Optional[int] = None,
) -> Tuple[np.ndarray, np.ndarray]:
    """Spherical K-Means on L2-normalized data (cosine geometry).

    - X: shape (N, D), will be L2-normalized internally.
    - Returns (centers, labels), where centers are unit L2-normalized.
    """
    assert n_clusters >= 1, "n_clusters must be >= 1"
    rng = np.random.RandomState(random_state)
    X = X.astype(np.float32, copy=True)
    # Normalize rows
    X_norm = np.linalg.norm(X, axis=1, keepdims=True).clip(min=1e-8)
    X = X / X_norm

    N, D = X.shape
    best_inertia = np.inf
    best_centers = None
    best_labels = None

    for _ in range(max(n_init, 1)):
        # Init centers by random samples
        idx = rng.choice(N, size=n_clusters, replace=False) if N >= n_clusters else np.arange(N)
        centers = X[idx].copy()
        # Ensure unit norm
        centers /= np.linalg.norm(centers, axis=1, keepdims=True).clip(min=1e-8)

        labels = np.zeros(N, dtype=np.int32)
        for _it in range(max_iter):
            # Assignment by cosine similarity (argmax dot product)
            sims = X @ centers.T  # (N, K)
            new_labels = sims.argmax(axis=1).astype(np.int32)
            if np.array_equal(new_labels, labels):
                break
            labels = new_labels

            # Update centers: mean then renormalize; handle empty clusters
            for k in range(n_clusters):
                mask = labels == k
                if not np.any(mask):
                    # Re-seed empty cluster to a random point
                    centers[k] = X[rng.randint(0, N)]
                else:
                    centers[k] = X[mask].mean(axis=0)
            centers /= np.linalg.norm(centers, axis=1, keepdims=True).clip(min=1e-8)

        # Inertia under cosine distance: sum_i (1 - max_k cos(x_i, c_k))
        sims = X @ centers.T
        inertia = float(np.sum(1.0 - sims.max(axis=1)))
        if inertia < best_inertia:
            best_inertia = inertia
            best_centers = centers.copy()
            best_labels = labels.copy()

    if best_centers is None:
        # Fallback single cluster
        best_centers = X.mean(axis=0, keepdims=True)
        best_centers /= np.linalg.norm(best_centers, axis=1, keepdims=True).clip(min=1e-8)
        best_labels = np.zeros(N, dtype=np.int32)
    return best_centers.astype(np.float32), best_labels.astype(np.int32)

# ------------------------------
# 0. Environment helpers
# ------------------------------

def in_colab() -> bool:
    """Detect whether the code is running inside a Google Colab runtime."""
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False


def try_mount_drive():
    """Best-effort mount of Google Drive when executing in Colab."""
    if in_colab():
        try:
            from google.colab import drive  # type: ignore
            drive.mount('/content/drive', force_remount=True)
            print('Google Drive mounted')
        except Exception as e:
            print(f'Drive mount failed (continuing): {e}')

# ------------------------------
# 1. Baseline repo + paths
# ------------------------------

BASELINE_CANDIDATES = [
    # Local workspace variants (with/without suffix / space)
    'Dataset-Distillation-via-Vision-Language-Category-Prototype',
]


def resolve_baseline_repo(project_root: str) -> str:
    """Resolve the baseline repository path across env vars, project_root, CWD, and Colab defaults."""
    # 1) Env override
    env_path = os.environ.get('BASELINE_REPO_PATH')
    if env_path and os.path.isdir(env_path):
        return env_path
    # 2) Under provided project_root
    for name in BASELINE_CANDIDATES:
        p = os.path.join(project_root, name)
        if os.path.isdir(p):
            return p
    # 3) Under current working directory
    for name in BASELINE_CANDIDATES:
        p = os.path.join(os.getcwd(), name)
        if os.path.isdir(p):
            return p
    # 4) Typical Colab project root fallback
    if in_colab():
        for name in BASELINE_CANDIDATES:
            p = os.path.join(DEFAULT_PROJECT_ROOT, name)
            if os.path.isdir(p):
                return p
    raise FileNotFoundError(
        'Baseline repo not found. Please place the repository folder in your project root.\n'
        f'Tried: {BASELINE_CANDIDATES} under {project_root}, CWD, and {DEFAULT_PROJECT_ROOT}'
    )

# ------------------------------
# 2. Configuration
# ------------------------------

class Config:
    """Container for experiment hyperparameters and resolved filesystem locations."""
    def __init__(self, base_seed: int = 0, project_root: Optional[str] = None):
        self.current_seed = base_seed

        # Dataset / protocol
        self.dataset = 'imagenet'
        self.spec = 'woof'
        self.nclass = 10
        self.resolution = 512
        self.output_image_size = 256  # saved image size for evaluation

        # Diffusion training (match baseline protocol)
        self.model_name = 'runwayml/stable-diffusion-v1-5'
        self.train_batch_size = 32
        self.gradient_accumulation_steps = 4
        self.mixed_precision = 'fp16'
        self.num_train_epochs = 8
        self.validation_epochs = 2
        self.learning_rate = 1e-5
        self.max_grad_norm = 1.0
        self.lr_scheduler = 'constant'
        self.lr_warmup_steps = 0
        self.use_ema = True

        # Prototype generation (IPC) and synthesis
        self.ipc = 10
        self.km_expand = 1
        self.contamination = 0.1
        self.guidance_scale = 10.5
        self.strength = 0.75
        self.negative_prompt = 'cartoon, anime, painting'

        # JVL-C params
        self.alpha = 0.5
        self.top_k_visual = 6  # Mean top-K representative visual latents

        # JVL-C alpha search controls (for experimentation)
        self.jvlc_global_alpha_search = False
        self.jvlc_global_alpha_min = 0.2
        self.jvlc_global_alpha_max = 0.8
        self.jvlc_global_alpha_steps = 7
        self.jvlc_alpha_align_weight = 0.0

        # CLIP model
        self.clip_model_name = 'openai/clip-vit-large-patch14'

        # Evaluation
        self.eval_repeat = 3

        # Paths
        # Follow baselinesetup.py layout: code/data under the default Colab project root; models on Drive
        self.base_path = project_root or (
            DEFAULT_PROJECT_ROOT if in_colab() else os.getcwd()
        )
        # Match baselinesetup.py: dataset under base_path/data/{train,val}
        # Data and outputs
        self.data_root = "/content/woofdata_local"
        self.train_dir = os.path.join(self.data_root, "train")
        self.val_dir = os.path.join(self.data_root, "val")

        self.outputs_root = os.path.join(self.base_path, "outputs")
        self.distilled_root = os.path.join(self.base_path, "distilled_images", f"seed_{self.current_seed}")
        self.prototype_root = os.path.join(self.base_path, "prototypes", f"seed_{self.current_seed}")


        self.class_file = "/content/jvl_vlpr/Dataset-Distillation-via-Vision-Language-Category-Prototype/03_distiilation/label-prompt/imagenet_woof_classes.txt"

        self.metadata_file = os.path.join(self.train_dir, 'metadata.jsonl')

        # Fine-tuned model output (exact path used in baselinesetup.py evaluation)
        models_dir_name = f"{PROJECT_BASENAME}_models"
        self.finetuned_model_path = (
            os.path.join(DEFAULT_DRIVE_MODELS_ROOT, 'ImageWoof_seed0')
            if in_colab()
            else os.path.join(self.base_path, models_dir_name, 'ImageWoof_seed0')
        )

        # Device
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'



    def set_seed(self, s: int):
        """Update the active seed and regenerate any seed-scoped output paths."""
        self.current_seed = s
        self.distilled_root = os.path.join(self.base_path, 'distilled_images', f'seed_{s}')
        self.prototype_root = os.path.join(self.base_path, 'prototypes', f'seed_{s}')

cfg = Config()
cfg.data_root = "/content/woofdata"   # resolves to /content/local_data

!readlink -f /content/woofdata  # Expect: /content/local_data

# ------------------------------
# 3. Dataset + metadata helpers
# ------------------------------

IMAGEWOOF_CLASS_MAPPING = {
    'n02086240': 'Shih-Tzu',
    'n02087394': 'Rhodesian ridgeback',
    'n02088364': 'Beagle',
    'n02089973': 'English foxhound',
    'n02093754': 'Australian terrier',
    'n02096294': 'Border terrier',
    'n02099601': 'Golden retriever',
    'n02105641': 'Old English sheepdog',
    'n02111889': 'Samoyed',
    'n02115641': 'Dingo'
}
_IMAGEWOOF_WNIDS = list(IMAGEWOOF_CLASS_MAPPING.keys())


def ensure_class_file(path: str):
    """Write the default WNID list to disk if the class file is missing."""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    if not os.path.isfile(path):
        with open(path, 'w') as f:
            f.write('\n'.join(_IMAGEWOOF_WNIDS) + '\n')
        print(f'Wrote class file: {path}')
    else:
        print(f'Using class file: {path}')


class IndexedImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target, index


def load_text_descriptions(metadata_file_path: str) -> Dict[str, str]:
    """Read metadata.jsonl-style file into a mapping of relative path -> text prompt."""
    descriptions: Dict[str, str] = {}
    with open(metadata_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            file_name = data['file_name']
            text = data['text']
            descriptions[file_name] = text
    print(f'Loaded {len(descriptions)} text descriptions from {metadata_file_path}')
    return descriptions


def get_transform_protocol(img_size: int) -> transforms.Compose:
    """Basic resize + tensor transform matching the ImageWoof preprocessing protocol."""
    return transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
    ])


def create_imagewoof_dataloader(data_dir: str, transform: transforms.Compose, batch_size: int = 10, num_workers: int = 4):
    """Create an indexed ImageFolder dataloader for the ImageWoof train split."""
    train_dir = os.path.join(data_dir, 'train')
    dataset = IndexedImageFolder(root=train_dir, transform=transform)

    # Build kwargs to avoid passing prefetch_factor when num_workers == 0
    dl_kwargs = dict(
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=False,
        pin_memory=True,
    )
    if num_workers and num_workers > 0:
        dl_kwargs.update(dict(persistent_workers=False, prefetch_factor=2))

    dataloader = DataLoader(dataset, **dl_kwargs)
    path_all = [path for path, _ in dataset.samples]
    print(f'Dataloader: {len(dataset)} images, classes={dataset.classes}')
    return dataloader, path_all, dataset.classes


# ------------------------------
# 4. Custom diffusion pipelines from baseline
# ------------------------------

def load_latents2img_pipeline(model_path: str):
    """Load StableDiffusionLatents2ImgPipeline from the patched diffusers install."""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    try:
        from diffusers import StableDiffusionLatents2ImgPipeline  # type: ignore
    except Exception as e:
        raise ImportError(
            'StableDiffusionLatents2ImgPipeline not importable.\n'
            'Run baselinesetup.py first to patch and reinstall diffusers.'
        ) from e

    # Use `dtype` (not deprecated `torch_dtype`)
    pipe = StableDiffusionLatents2ImgPipeline.from_pretrained(model_path, torch_dtype=dtype).to(device)

    # Memory savers: identical outputs, lower VRAM
    pipe.enable_attention_slicing()
    pipe.vae.enable_slicing()
    pipe.vae.enable_tiling()

    pipe.set_progress_bar_config(disable=True)
    print('Loaded Latents2Img from patched diffusers (memory-optimized)')
    return pipe

# ------------------------------
# 5. JVL algorithms (clustering + helpers)
# ------------------------------

class JVLClustering:
    """Phase 1: Joint Vision-Language Clustering in CLIP space"""
    def __init__(self, alpha: float = 0.5, device: str = 'cuda', diag_checks: bool = True, random_state: int = 0,
                 # Mathematical per-cluster alpha optimization (enabled by default)
                 alpha_optimize: bool = True,
                 alpha_opt_min: float = 0.2,
                 alpha_opt_max: float = 0.8,
                 alpha_opt_steps: int = 13,
                 alpha_reg: float = 0.6,
                 top_k_visual: int = 3,
                 sim_high_threshold: float = 0.85,
                 sim_low_threshold: float = 0.65,
                 alpha_when_high: float = 0.3,
                 alpha_when_low: float = 0.7):
        self.alpha = alpha
        self.device = device
        self.diag_checks = diag_checks
        self.random_state = random_state
        # Adaptive-alpha (threshold) params
        self.sim_high_threshold = sim_high_threshold
        self.sim_low_threshold = sim_low_threshold
        self.alpha_when_high = alpha_when_high
        self.alpha_when_low = alpha_when_low
        # Alpha optimization params
        self.alpha_optimize = alpha_optimize
        self.alpha_opt_min = alpha_opt_min
        self.alpha_opt_max = alpha_opt_max
        self.alpha_opt_steps = alpha_opt_steps
        self.alpha_reg = alpha_reg
        self.top_k_visual = top_k_visual

        print('Loading CLIP model...')
        self.clip_model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')
        self.clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14')
        self.clip_model = self.clip_model.to(device)
        self.clip_model.eval()

        print('Loading VAE model...')
        self.vae = AutoencoderKL.from_pretrained(
            'runwayml/stable-diffusion-v1-5',
            subfolder='vae',
            torch_dtype=torch.float16 if device == 'cuda' else torch.float32,
        )
        self.vae = self.vae.to(device)
        self.vae.eval()
        if hasattr(self.vae, "enable_slicing"): self.vae.enable_slicing()
        if hasattr(self.vae, "enable_tiling"): self.vae.enable_tiling()

        print(f'JVL-C initialized (alpha={alpha})')

    def normalize(self, v: torch.Tensor) -> torch.Tensor:
        """L2-normalize embeddings along the last dimension."""
        return v / torch.norm(v, dim=-1, keepdim=True)

    def extract_vae_latents(self, images: torch.Tensor, batch_size: int = 16) -> torch.Tensor:
        """Encode a batch of images into scaled VAE latents while controlling memory usage."""
        latents_out = []
        with torch.inference_mode():
            for i in range(0, images.size(0), batch_size):
                chunk = images[i : i + batch_size]
                chunk = chunk.to(device=self.vae.device, dtype=self.vae.dtype, non_blocking=True)
                chunk = 2.0 * chunk - 1.0
                dist = self.vae.encode(chunk).latent_dist
                lat = dist.sample()
                lat = lat * self.vae.config.scaling_factor
                latents_out.append(lat)
                del chunk, dist, lat
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
        return torch.cat(latents_out, dim=0)

    def decode_latents(self, latents: torch.Tensor) -> torch.Tensor:
        """Decode VAE latents back to image space for evaluation outputs."""
        with torch.inference_mode():
            latents = latents.to(device=self.vae.device, dtype=self.vae.dtype, non_blocking=True)
            latents = latents / self.vae.config.scaling_factor
            images = self.vae.decode(latents).sample
            images = (images + 1.0) / 2.0
        return torch.clamp(images, 0.0, 1.0)



    def extract_clip_embeddings(self, images: torch.Tensor, texts: List[str]) -> Tuple[torch.Tensor, torch.Tensor]:
        """Compute normalized CLIP image and text embeddings for a batch of samples."""
        with torch.no_grad():
            image_inputs = self.clip_processor(
                images=[transforms.ToPILImage()(img) for img in images],
                return_tensors='pt',
                padding=True,
            ).to(self.device)

            text_inputs = self.clip_processor(
                text=texts,
                return_tensors='pt',
                padding=True,
                truncation=True,
            ).to(self.device)

            image_features = self.clip_model.get_image_features(**image_inputs)
            text_features = self.clip_model.get_text_features(**text_inputs)
            image_features = self.normalize(image_features)
            text_features = self.normalize(text_features)
        return image_features, text_features

    def create_fused_embeddings(self, image_features: torch.Tensor, text_features: torch.Tensor, alpha: Optional[float] = None) -> torch.Tensor:
        """Blend image/text embeddings with the configured alpha and renormalize."""
        a = self.alpha if alpha is None else alpha
        fused = a * image_features + (1 - a) * text_features
        return self.normalize(fused)

    def select_text_prototype_adaptive(self,
        cluster_indices: np.ndarray,
        descriptions: List[str],
        best_idx_local: int,
        avg_sim_local_score: float,
        avg_sim: float,
        alpha_used: float,
    ) -> str:
        """Pick the most central text description (hubness-aware) for a cluster."""
        # Choose the text whose fused vector has the highest
        # average cosine similarity to all other fused vectors in the cluster.
        # This favors a representative (central) text and reduces chance of hub outliers.
        # We reuse the index selected during visual prototype selection (which uses the same logic).

        chosen_global_idx = int(cluster_indices[best_idx_local])
        text_prototype = descriptions[chosen_global_idx]
        print(
            f"  Selected text prototype (hubness avg_sim={avg_sim_local_score:.4f}, "
            f"cluster_avg_imgtxt={avg_sim:.4f}, α*={alpha_used:.2f})"
        )
        return text_prototype

    def _optimize_alpha_for_cluster(
        self,
        img_feats_c: torch.Tensor,
        txt_feats_c: torch.Tensor,
        tau_pos: Optional[torch.Tensor] = None,
        tau_neg_stack: Optional[torch.Tensor] = None,
        alpha_prior: Optional[float] = None,
        lambda_prior: Optional[float] = None,
        w_intra: float = 1.0,
        w_align: float = 0.7,
        w_margin: float = 0.2,
        w_cluster_sep: float = 1.0,  # High urgency: Weight for inter-cluster separation
        other_cluster_centers: Optional[torch.Tensor] = None,  # NEW: Centroids of other clusters in same class
    ) -> Tuple[float, Dict[str, float]]:
        """Directional Fisher objective for per-cluster alpha selection."""
        device = img_feats_c.device
        a_min = float(self.alpha_opt_min)
        a_max = float(self.alpha_opt_max)
        lam = self.alpha_reg if lambda_prior is None else lambda_prior
        tau_pos = tau_pos.to(device) if tau_pos is not None else None
        if tau_neg_stack is not None:
            tau_neg_stack = tau_neg_stack.to(device)

        # --- Dynamic Weighting Probe (Per-Cluster) ---
        probe_steps = 5
        probe_alphas = torch.linspace(a_min, a_max, steps=probe_steps).tolist()
        intra_vals = []
        align_vals = []
        margin_vals = []

        for a in probe_alphas:
            fused = a * img_feats_c + (1.0 - a) * txt_feats_c
            fused = fused / fused.norm(dim=-1, keepdim=True).clamp_min(1e-8)
            mu = fused.mean(dim=0, keepdim=True)
            mu = mu / mu.norm(dim=-1, keepdim=True).clamp_min(1e-8)
            intra = (1.0 - (fused @ mu.t()).squeeze(-1)).mean().item()
            align = float(torch.dot(mu.squeeze(0), tau_pos).item()) if tau_pos is not None else 0.0

            if tau_neg_stack is not None and tau_neg_stack.numel() > 0:
                sims = (mu @ tau_neg_stack.t()).squeeze(0)
                k = min(3, sims.numel())
                worst_neg = torch.topk(sims, k=k, largest=True).values.mean().item()
                margin = 1.0 - float(worst_neg)
            else:
                margin = 1.0 # Default to perfect margin if no negatives

            intra_vals.append(intra)
            align_vals.append(align)
            margin_vals.append(margin)

        intra_med = float(np.median(intra_vals))
        align_med = float(np.median(align_vals))
        margin_med = float(np.median(margin_vals))

        gap_intra = max(0.0, intra_med)
        gap_align = max(0.0, 1.0 - align_med)
        gap_margin = max(0.0, 1.0 - margin_med) # Perfect margin = 1.0 (uncorrelated)

        w_intra_dyn = w_intra * gap_intra
        w_align_dyn = w_align * gap_align
        w_margin_dyn = w_margin * gap_margin

        # Normalize to sum to 1.0 (or similar scale)
        total_w = w_intra_dyn + w_align_dyn + w_margin_dyn
        if total_w < 1e-6:
             w_intra_final, w_align_final, w_margin_final = w_intra, w_align, w_margin
        else:
             w_intra_final = w_intra_dyn / total_w
             w_align_final = w_align_dyn / total_w
             w_margin_final = w_margin_dyn / total_w

        # print(f"  [Cluster] DynWeights: Intra={w_intra_final:.2f}, Align={w_align_final:.2f}, Margin={w_margin_final:.2f}")
        # ---------------------------------------------

        # --- Inter-Cluster Separation (NEW) ---
        # If we have other cluster centers, compute gap for cluster separation
        if other_cluster_centers is not None and other_cluster_centers.numel() > 0:
            other_cluster_centers = other_cluster_centers.to(device)
            gap_cluster_sep_vals = []
            for a in probe_alphas:
                fused = a * img_feats_c + (1.0 - a) * txt_feats_c
                fused = fused / fused.norm(dim=-1, keepdim=True).clamp_min(1e-8)
                mu = fused.mean(dim=0, keepdim=True)
                mu = mu / mu.norm(dim=-1, keepdim=True).clamp_min(1e-8)
                # Similarity to other cluster centers (lower is better = more separated)
                sims_to_others = (mu @ other_cluster_centers.t()).squeeze(0)
                worst_cluster_sim = sims_to_others.max().item() if sims_to_others.numel() > 0 else 0.0
                cluster_margin = 1.0 - worst_cluster_sim
                # Hinge Loss for Dynamic Weighting: Cap gap at m_div
                m_div = 0.4
                gap_cluster_sep_vals.append(max(0.0, m_div - min(cluster_margin, m_div)))  # ← INSIDE the loop

            # After the loop - calculate median and weight
            gap_cluster_sep = float(np.median(gap_cluster_sep_vals))
            w_cluster_sep_dyn = w_cluster_sep * (gap_cluster_sep / m_div)
        else:
            gap_cluster_sep = 0.0
            w_cluster_sep_dyn = 0.0

        # Re-normalize with cluster separation weight
        total_w = w_intra_final + w_align_final + w_margin_final + w_cluster_sep_dyn
        if total_w > 1e-6:
            w_intra_final = w_intra_final / total_w
            w_align_final = w_align_final / total_w
            w_margin_final = w_margin_final / total_w
            w_cluster_sep_final = w_cluster_sep_dyn / total_w
        else:
            w_cluster_sep_final = 0.0
        # ---------------------------------------

        def cluster_objective(a: float) -> float:
            fused = a * img_feats_c + (1.0 - a) * txt_feats_c
            fused = fused / fused.norm(dim=-1, keepdim=True).clamp_min(1e-8)
            mu = fused.mean(dim=0, keepdim=True)
            mu = mu / mu.norm(dim=-1, keepdim=True).clamp_min(1e-8)
            cos = (fused @ mu.t()).squeeze(-1)
            intra = (1.0 - cos).mean().item()
            align = float(torch.dot(mu.squeeze(0), tau_pos).item()) if tau_pos is not None else 0.0
            if tau_neg_stack is not None and tau_neg_stack.numel() > 0:
                sims = (mu @ tau_neg_stack.t()).squeeze(0)
                k = min(3, sims.numel())
                worst_neg = torch.topk(sims, k=k, largest=True).values.mean().item()
                margin = 1.0 - float(worst_neg)
            else:
                margin = 0.0

            # Inter-cluster separation (NEW)
            if other_cluster_centers is not None and other_cluster_centers.numel() > 0:
                sims_to_others = (mu @ other_cluster_centers.t()).squeeze(0)
                worst_cluster_sim = sims_to_others.max().item() if sims_to_others.numel() > 0 else 0.0
                cluster_margin = 1.0 - worst_cluster_sim
                # Hinge Loss: Cap the benefit of separation at m_div (e.g., 0.4)
                # Once clusters are separated by > 0.4 (approx 72 degrees), no further gain.
                m_div = 0.4
                cluster_sep = min(cluster_margin, m_div)
            else:
                cluster_sep = 0.0

            score = (-w_intra_final * intra) + (w_align_final * align) + (w_margin_final * margin) + (w_cluster_sep_final * cluster_sep)
            if alpha_prior is not None and lam > 0.0:
                score -= lam * ((a - alpha_prior) ** 2)
            return float(score)

        alpha_star, _ = golden_section_maximize(cluster_objective, lo=a_min, hi=a_max, tol=1e-3, max_iter=32)

        fused_star = alpha_star * img_feats_c + (1.0 - alpha_star) * txt_feats_c
        fused_star = fused_star / fused_star.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        mu_star = fused_star.mean(dim=0, keepdim=True)
        mu_star = mu_star / mu_star.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        cos_star = (fused_star @ mu_star.t()).squeeze(-1)
        intra_star = (1.0 - cos_star).mean().item()
        align_star = float(torch.dot(mu_star.squeeze(0), tau_pos).item()) if tau_pos is not None else float('nan')
        if tau_neg_stack is not None and tau_neg_stack.numel() > 0:
            sims = (mu_star @ tau_neg_stack.t()).squeeze(0)
            k = min(3, sims.numel())
            worst_neg = torch.topk(sims, k=k, largest=True).values.mean().item()
            margin_star = 1.0 - float(worst_neg)
        else:
            margin_star = float('nan')
        metrics = {"intra": intra_star, "align": align_star, "margin": margin_star}
        return float(alpha_star), metrics

    def derive_prototypes_jvl_c(
        self,
        vae_latents: torch.Tensor,
        clip_img_embeds: torch.Tensor,
        clip_txt_embeds: torch.Tensor,
        descriptions: List[str],
        ipc: int,
        contamination: float = 0.1,
        alpha_override: Optional[float] = None,
        tau_pos: Optional[torch.Tensor] = None,
        tau_neg_stack: Optional[torch.Tensor] = None,
        align_weight: float = 0.7,
        wnid: Optional[str] = None,
    ) -> Tuple[List[Dict], np.ndarray, torch.Tensor]:
        """Create JVL-C prototypes for a single class, optionally refining alpha and filtering outliers."""
        print(f'\nJVL-C: IPC={ipc}, contamination={contamination}')
        alpha_base = self.alpha if alpha_override is None else float(alpha_override)
        fused_embeds = self.create_fused_embeddings(clip_img_embeds, clip_txt_embeds, alpha=alpha_base)
        print(f'Fused embeddings: {tuple(fused_embeds.shape)}')

        if contamination > 0:
            print(f'Outlier detection (LOF) with contamination={contamination}')
            clf = LocalOutlierFactor(n_neighbors=10, contamination=contamination)
            X = fused_embeds.detach().cpu().numpy()
            y_pred = clf.fit_predict(X)
            inliers = y_pred == 1
            print(f'Inliers: {np.sum(inliers)}/{len(inliers)}')
            fused_embeds = fused_embeds[inliers]
            vae_latents = vae_latents[inliers]
            clip_img_embeds = clip_img_embeds[inliers]
            clip_txt_embeds = clip_txt_embeds[inliers]
            descriptions = [descriptions[i] for i in range(len(descriptions)) if inliers[i]]
        else:
            inliers = np.ones(len(fused_embeds), dtype=bool)

        if len(fused_embeds) < ipc:
            print(f'Not enough samples after filtering: {len(fused_embeds)} < {ipc}. Reducing IPC.')
            ipc = len(fused_embeds)

        centers_np, labels_np = spherical_kmeans(
            fused_embeds.detach().cpu().numpy(),
            n_clusters=int(ipc),
            n_init=10,
            max_iter=50,
            random_state=self.random_state,
        )
        cluster_centers = torch.tensor(centers_np, dtype=torch.float32, device=self.device)
        labels = labels_np

        # --- Pre-compute initial cluster centroids for inter-cluster separation ---
        # These are the centroids in the fused space (with base alpha) for each cluster
        initial_cluster_centroids = []
        for k in range(ipc):
            cluster_mask = labels == k
            if cluster_mask.sum() > 0:
                cluster_fused = fused_embeds[cluster_mask]
                centroid = cluster_fused.mean(dim=0)
                centroid = centroid / centroid.norm().clamp_min(1e-8)
                initial_cluster_centroids.append(centroid)
            else:
                initial_cluster_centroids.append(None)
        # Convert to tensor, filtering out None (empty clusters)
        valid_centroids = [c for c in initial_cluster_centroids if c is not None]
        if len(valid_centroids) > 1:
            all_cluster_centroids = torch.stack(valid_centroids)
        else:
            all_cluster_centroids = None
        print(f"Computed {len(valid_centroids)} cluster centroids for inter-cluster separation")

        # --- Visualization for English Springer (n02102040) ---
        if wnid == 'n02102040':
            try:
                print(f"Generating visualizations for {wnid} (English Springer)...")
                exp_dir = os.path.join('/content/drive/MyDrive' if os.path.isdir('/content/drive/MyDrive') else '.', 'experiment')
                os.makedirs(exp_dir, exist_ok=True)

                # UMAP with Cosine Metric
                print(f"  Running UMAP (cosine) for {wnid}...")
                X_np = fused_embeds.detach().cpu().numpy()

                reducer = umap.UMAP(
                    n_neighbors=15,      # Controls local structure (15-50 is good)
                    min_dist=0.1,        # Controls how tightly points are packed (0.1 is tight)
                    metric='cosine',     # <--- CRITICAL: Uses spherical angles
                    random_state=self.random_state
                )
                X_embedded = reducer.fit_transform(X_np)

                plt.figure(figsize=(12, 10))
                # Plot all points colored by cluster
                scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=labels, cmap='tab10', alpha=0.6, s=30)
                plt.colorbar(scatter, label='Cluster ID')
                plt.title(f'UMAP (Cosine) of Fused Space for {wnid} (English Springer)')
                plt.savefig(os.path.join(exp_dir, f'{wnid}_umap_clusters.png'))
                plt.close()

                # We will collect selected indices to highlight them later
                selected_indices_global = []
            except Exception as e:
                print(f"Visualization failed: {e}")
                import traceback
                traceback.print_exc()
        # ------------------------------------------------------

        prototypes: List[Dict] = []
        for k in range(ipc):
            cluster_indices = np.where(labels == k)[0]
            if len(cluster_indices) == 0:
                print(f'Empty cluster {k}')
                continue
            cluster_vae_latents = vae_latents[cluster_indices]

            # Build "other cluster centroids" for this cluster (exclude self)
            if all_cluster_centroids is not None and len(valid_centroids) > 1:
                # Filter out the current cluster's centroid
                other_centroids_list = [initial_cluster_centroids[j] for j in range(ipc) if j != k and initial_cluster_centroids[j] is not None]
                if len(other_centroids_list) > 0:
                    other_cluster_centers = torch.stack(other_centroids_list)
                else:
                    other_cluster_centers = None
            else:
                other_cluster_centers = None

            # Compute per-cluster alpha via optimization (or threshold fallback)
            img_feats_c = clip_img_embeds[cluster_indices]
            txt_feats_c = clip_txt_embeds[cluster_indices]
            alpha_c, metrics = self._optimize_alpha_for_cluster(
                img_feats_c,
                txt_feats_c,
                tau_pos=tau_pos,
                tau_neg_stack=tau_neg_stack,
                alpha_prior=alpha_base,
                lambda_prior=self.alpha_reg,
                w_align=align_weight,
                other_cluster_centers=other_cluster_centers,
            )
            avg_sim = torch.mean(torch.sum(img_feats_c * txt_feats_c, dim=-1)).item()

            # Build adaptive fused space for selection
            cluster_fused_adapt = self.create_fused_embeddings(img_feats_c, txt_feats_c, alpha=alpha_c)

            # Hubness-based selection (Average Similarity)
            # Replaces the old distance-to-mean logic which was being overwritten anyway
            k_visual = max(1, min(self.top_k_visual, len(cluster_indices)))

            if cluster_fused_adapt.size(0) == 1:
                top_local_indices = torch.tensor([0], device=cluster_fused_adapt.device)
                best_idx_local = 0
                best_hub_score = 1.0
            else:
                sim = cluster_fused_adapt @ cluster_fused_adapt.t()
                n = cluster_fused_adapt.size(0)
                # Exclude self-similarity
                row_sum = sim.sum(dim=1) - torch.ones(n, device=sim.device)
                avg_sim_local = row_sum / max(1, n - 1)

                # Select top-k for visual
                top_local_indices = torch.topk(avg_sim_local, k_visual).indices

                # Best one for text
                best_hub_score, best_idx_local = torch.max(avg_sim_local, dim=0)
                best_hub_score = best_hub_score.item()
                best_idx_local = int(best_idx_local.item())

            top_global_indices = cluster_indices[top_local_indices.detach().cpu().numpy()]
            top_latents = vae_latents[top_global_indices]
            image_prototype_Zc = torch.mean(top_latents, dim=0)

            # Store top-1 visual latent separately for cleaner visualization
            best_global_idx = cluster_indices[best_idx_local]
            top1_visual_latent = vae_latents[best_global_idx]

            print(f"  Selected mean of top-{k_visual} visual prototypes (hubness-based)")

            text_prototype_Tc = self.select_text_prototype_adaptive(
                cluster_indices=cluster_indices,
                descriptions=descriptions,
                best_idx_local=best_idx_local,
                avg_sim_local_score=best_hub_score,
                avg_sim=avg_sim,
                alpha_used=alpha_c,
            )

            # --- Collect selected indices for visualization ---
            if wnid == 'n02102040':
                 # top_global_indices are the indices in the original (inlier) set
                 if isinstance(top_global_indices, torch.Tensor):
                     selected_indices_global.extend(top_global_indices.detach().cpu().tolist())
                 else:
                     selected_indices_global.extend(top_global_indices.tolist())
            # ------------------------------------------------
            prototypes.append({
                'image_prototype': image_prototype_Zc,
                'top1_visual_latent': top1_visual_latent,  # For cleaner visualization
                'text_prototype': text_prototype_Tc,
                'cluster_size': int(len(cluster_indices)),
                'cluster_center': cluster_centers[k],
                'alpha_used': alpha_c,
                'avg_imgtxt_sim': avg_sim,
            })
        print(f'JVL-C complete: {len(prototypes)} prototypes')

        # --- Finalize Visualization for English Springer ---
        if wnid == 'n02102040':
            try:
                # Highlight selected points on UMAP
                plt.figure(figsize=(12, 10))
                scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=labels, cmap='tab10', alpha=0.4, s=30, label='All Points')

                # Highlight selected
                selected_mask = np.zeros(len(X_embedded), dtype=bool)
                selected_mask[selected_indices_global] = True

                plt.scatter(X_embedded[selected_mask, 0], X_embedded[selected_mask, 1],
                           c='red', marker='*', s=200, edgecolors='black', label='Selected Prototypes')

                plt.legend()
                plt.title(f'Selected Prototypes for {wnid} on UMAP')
                plt.savefig(os.path.join(exp_dir, f'{wnid}_selected_prototypes_umap.png'))
                plt.close()

                # Plot Metrics per Cluster
                cluster_ids = [k for k in range(len(prototypes))]
                intras = [1.0 - p.get('avg_imgtxt_sim', 0) for p in prototypes] # Approx intra as 1-sim? No, use recorded stats if available.
                # The prototype dict has 'avg_imgtxt_sim'. Let's use that as a proxy for alignment/compactness.
                # Or we can compute exact intra/inter for the plot.
                # User asked for "intra, inter and distilled images with textual prototypes alignment per class and per cluster"
                # We have 'avg_imgtxt_sim' (alignment).

                aligns = [p['avg_imgtxt_sim'] for p in prototypes]
                alphas = [p['alpha_used'] for p in prototypes]

                fig, ax1 = plt.subplots(figsize=(10, 6))
                ax1.set_xlabel('Cluster ID')
                ax1.set_ylabel('Alignment (Img-Txt)', color='tab:blue')
                ax1.bar(cluster_ids, aligns, color='tab:blue', alpha=0.6, label='Alignment')
                ax1.tick_params(axis='y', labelcolor='tab:blue')

                ax2 = ax1.twinx()
                ax2.set_ylabel('Alpha Used', color='tab:red')
                ax2.plot(cluster_ids, alphas, color='tab:red', marker='o', label='Alpha')
                ax2.tick_params(axis='y', labelcolor='tab:red')

                plt.title(f'Per-Cluster Metrics for {wnid}')
                plt.savefig(os.path.join(exp_dir, f'{wnid}_cluster_metrics.png'))
                plt.close()

                # --- New: Visual Grid of Prototypes with Metrics ---
                # Show TOP-1 visual (single best image, not blurry mean) + text prototype
                print(f"  Generating prototype grid for {wnid}...")
                # Use top1_visual_latent for cleaner visualization instead of the mean
                top1_latents = torch.stack([p['top1_visual_latent'] for p in prototypes])
                # Decode needs (B, 4, H, W)
                with torch.no_grad():
                    decoded_imgs = self.decode_latents(top1_latents)  # Returns (B, 3, H, W) in [0, 1]

                n_protos = len(prototypes)
                cols = 5
                rows = (n_protos + cols - 1) // cols

                # Increased height per row to accommodate wrapped text below images
                fig_height = 7 * rows + 3  # More vertical space per row
                fig_width = 5 * cols
                fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))
                # Flatten axes for easy iteration; handle single row case
                if rows == 1:
                    axes = [axes] if cols == 1 else list(axes)
                else:
                    axes = axes.flatten()

                for i, (img, p) in enumerate(zip(decoded_imgs, prototypes)):
                    ax = axes[i]
                    # img is (3, H, W), convert to (H, W, 3) for matplotlib
                    # Ensure float32 dtype - VAE may return float16 which matplotlib's imshow doesn't support
                    img_np = img.permute(1, 2, 0).cpu().float().numpy()
                    ax.imshow(img_np)

                    # Metrics
                    align = p.get('avg_imgtxt_sim', 0.0)
                    alpha = p.get('alpha_used', 0.0)
                    sz = p.get('cluster_size', 0)
                    text_proto = p.get('text_prototype', '')

                    # Title above image (metrics only)
                    ax.set_title(f"C{i} | α={alpha:.2f} | Align={align:.3f} | Sz={sz}", fontsize=10, fontweight='bold')
                    ax.axis('off')

                    # Text prototype BELOW image with word wrapping
                    # Wrap text at ~50 chars per line for readability
                    import textwrap
                    wrapped_text = textwrap.fill(f'\"{text_proto}\"', width=45)
                    ax.text(0.5, -0.02, wrapped_text, transform=ax.transAxes, fontsize=8,
                            ha='center', va='top', wrap=True,
                            bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.8))

                # Turn off unused axes
                for j in range(n_protos, len(axes)):
                    axes[j].axis('off')

                plt.suptitle(f"Top-1 Visual Prototypes for {wnid}\n(Best hub image per cluster + Text Prototype)", fontsize=16, fontweight='bold', y=0.99)
                plt.subplots_adjust(top=0.93, bottom=0.02, hspace=0.6, wspace=0.3)  # More vertical space between rows
                plt.savefig(os.path.join(exp_dir, f'{wnid}_prototypes_grid.png'), dpi=150, bbox_inches='tight')
                plt.close()
                # ---------------------------------------------------

                print(f"Saved English Springer visualizations to {exp_dir}")
            except Exception as e:
                print(f"Final visualization failed: {e}")
        # ---------------------------------------------------

        return prototypes, inliers, clip_txt_embeds


# ------------------------------
# 6. Orchestration: distill + save images
# ------------------------------

def save_images_for_evaluation(latents: torch.Tensor, class_wnid: str, phase: str, clustering: JVLClustering, out_root: str) -> str:
    """Decode and persist a batch of latents for a class/phase, returning the phase root."""
    save_dir = os.path.join(out_root, f'distilled_images_{phase}', class_wnid)
    os.makedirs(save_dir, exist_ok=True)
    with torch.inference_mode():
        images = clustering.decode_latents(latents)
        for idx, img in enumerate(images):
            save_image(img, os.path.join(save_dir, f'{idx}.png'))
    return os.path.join(out_root, f'distilled_images_{phase}')


def run_jvl_distillation(config: Config, baseline_repo: str, finetuned_model_path: Optional[str] = None) -> str:
    """Full JVL-C distillation loop across classes using the provided configuration."""
    torch.manual_seed(config.current_seed)
    np.random.seed(config.current_seed)

    ensure_class_file(config.class_file)

    # NLTK needs data for tokenization if used elsewhere
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords', quiet=True)

    # Load dataset + text descriptions
    transform = get_transform_protocol(config.resolution)
    dataloader, path_all, dataset_classes = create_imagewoof_dataloader(config.data_root, transform, batch_size=10)
    descriptions_map = load_text_descriptions(config.metadata_file)

    # Group by WNID
    class_ids = [line.strip() for line in open(config.class_file, 'r') if line.strip()]
    class_data = {wnid: {'images': [], 'labels': [], 'paths': []} for wnid in class_ids}
    for images, labels, indices in tqdm(dataloader, desc='Collecting data'):
        paths = [path_all[i] for i in indices]
        for img, label, path in zip(images, labels, paths):
            wnid = dataset_classes[label.item()]
            class_data[wnid]['images'].append(img)
            class_data[wnid]['labels'].append(label)
            class_data[wnid]['paths'].append(path)

    # Initialize clustering utilities
    jvlc = JVLClustering(
        alpha=config.alpha,
        device=config.device,
        diag_checks=True,
        random_state=config.current_seed,
        top_k_visual=config.top_k_visual,
    )

    prompt_name_tfs = build_prompt_ensemble_name_tfs(class_ids, jvlc.clip_model, jvlc.clip_processor, config.device)
    tau_neg_cache: Dict[str, Optional[torch.Tensor]] = {}
    for wnid in class_ids:
        negs = [prompt_name_tfs[other] for other in class_ids if other != wnid and other in prompt_name_tfs]
        tau_neg_cache[wnid] = torch.stack(negs, dim=0) if negs else None

    model_path = finetuned_model_path or config.finetuned_model_path
    if not os.path.isdir(model_path):
        raise FileNotFoundError(
            f"Fine-tuned model not found at {model_path}.\n"
            "Please run the fine-tuning step in baselinesetup.py to create it, or pass finetuned_model_override."
        )
    l2i_pipe = load_latents2img_pipeline(model_path=model_path)

    print("Latents2Img (created) device:",
    next(l2i_pipe.unet.parameters()).device,
    "dtype:", next(l2i_pipe.unet.parameters()).dtype)

        # Faster on big GPUs (disable slicing)
    try:
        l2i_pipe.disable_attention_slicing()
    except Exception:
        pass

    # Try xFormers attention if available (identical outputs)
    try:
        l2i_pipe.enable_xformers_memory_efficient_attention()
    except Exception as e:
        print("xFormers not available:", e)



    # Output roots
    os.makedirs(config.distilled_root, exist_ok=True)
    jvl_c_root = os.path.join(config.distilled_root, 'distilled_images_jvl_c')
    os.makedirs(jvl_c_root, exist_ok=True)




    # Process each class
    for wnid in class_ids:
        imgs = class_data[wnid]['images']
        if len(imgs) == 0:
            print(f'No images for class {wnid}')
            continue

        class_tensor = torch.stack(imgs)
        # Build texts per-image from metadata
        texts = []
        for p in class_data[wnid]['paths']:
            # path ends with .../train/<WNID>/<file>
            rel = os.path.join(wnid, os.path.basename(p))
            # Some metadata uses filenames without _WNID suffix
            alt = rel.replace(f'_{wnid}.JPEG', '.JPEG')
            text = descriptions_map.get(rel) or descriptions_map.get(alt) or ''
            texts.append(text)

        # 1) Extract features
        vae_latents = jvlc.extract_vae_latents(class_tensor)
        clip_img, clip_txt = jvlc.extract_clip_embeddings(class_tensor, texts)

        tau_pos = prompt_name_tfs[wnid]
        tau_neg_stack = tau_neg_cache.get(wnid)
        r_c = torch.mean(torch.sum(clip_img * clip_txt, dim=-1)).item()
        g = 1.0 / (1.0 + math.exp(-(r_c - 0.25) / 0.05))
        align_scale = 0.3 + 0.7 * g
        a_lo = 0.45 if r_c < 0.25 else config.jvlc_global_alpha_min
        a_hi = config.jvlc_global_alpha_max
        a_lo = min(a_hi, max(config.jvlc_global_alpha_min, a_lo))
        alpha_class = refine_alpha_per_class(
            alpha_global=config.alpha,
            wnid=wnid,
            img_feats=clip_img,
            txt_feats=clip_txt,
            tau_c=tau_pos,
            align_weight=align_scale,
            lam=0.6,
            a_min=a_lo,
            a_max=a_hi,
            device=config.device,
        )

        # 2) Prototypes via JVL-C
        prototypes, inliers, _ = jvlc.derive_prototypes_jvl_c(
            vae_latents,
            clip_img,
            clip_txt,
            texts,
            ipc=config.ipc,
            contamination=config.contamination,
            alpha_override=alpha_class,
            tau_pos=tau_pos,
            tau_neg_stack=tau_neg_stack,
            align_weight=align_scale,
            wnid=wnid,
        )

        if len(prototypes) == 0:
            print(f'No prototypes for class {wnid}')
            continue

        # Ensure pipeline on correct device and dtype
        # device = config.device
        # dtype = torch.float16 if device == 'cuda' else torch.float32
        # Ensure pipeline is on a valid device/dtype
        try:
            l2i_pipe.to(torch_device=config.device,
                        torch_dtype=(torch.float16 if config.device == 'cuda' else torch.float32))
        except TypeError:
            l2i_pipe.to(config.device)

        # Authoritative device/dtype from UNet
        pipe_device = next(l2i_pipe.unet.parameters()).device
        pipe_dtype  = next(l2i_pipe.unet.parameters()).dtype

        # Timesteps (some versions accept device)
        try:
            l2i_pipe.scheduler.set_timesteps(50, device=pipe_device)
        except TypeError:
            l2i_pipe.scheduler.set_timesteps(50)

        # Optional fallback (older diffusers): move buffers to the same device
        if hasattr(l2i_pipe.scheduler, "alphas_cumprod") and torch.is_tensor(l2i_pipe.scheduler.alphas_cumprod):
            l2i_pipe.scheduler.alphas_cumprod = l2i_pipe.scheduler.alphas_cumprod.to(pipe_device)
        if hasattr(l2i_pipe.scheduler, "timesteps") and torch.is_tensor(l2i_pipe.scheduler.timesteps):
            l2i_pipe.scheduler.timesteps = l2i_pipe.scheduler.timesteps.to(pipe_device)

        # One generator on the pipeline device (optionally seed)
        gen = torch.Generator(device=pipe_device)

        # Optional: show progress bar inside SD calls
        l2i_pipe.set_progress_bar_config(disable=False)


        # 3) Generate synthesized initial latents via Latents2Img (output_type="latent")
        import time

        initial_latents_list = []

        for idx, proto in enumerate(prototypes, 1):
            Zc = proto['image_prototype'].unsqueeze(0).to(device=pipe_device, dtype=pipe_dtype)
            Tc = proto['text_prototype']

            # Optional progress
            def _cb(step, ts, _=None):
                if step % 5 == 0 or step == 49:
                    print(f"  [{wnid}] SD proto {idx}/{len(prototypes)} step {step+1}/50 (t={int(ts)})")

            t0 = time.time()
            out = l2i_pipe(
                prompt=Tc,
                latents=Zc,
                is_init=True,
                strength=config.strength,
                guidance_scale=config.guidance_scale,
                num_inference_steps=50,
                negative_prompt=config.negative_prompt,
                output_type='latent',
                generator=gen,
                callback=_cb, callback_steps=5,   # optional
            )
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            print(f"  [{wnid}] proto {idx} took {time.time()-t0:.1f}s")

            Z_init = out.images if isinstance(out.images, torch.Tensor) else torch.stack(out.images)
            initial_latents_list.append(Z_init.squeeze(0))

        initial_latents = torch.stack(initial_latents_list)

        # Save JVL-C images (pre-refinement)
        save_images_for_evaluation(initial_latents, wnid, 'jvl_c', jvlc, config.distilled_root)

        # Safe per-class cleanup
        objs = [
            class_tensor, vae_latents, clip_img, clip_txt,
            prototypes, initial_latents_list, initial_latents, texts
        ]
        free_memory(*objs)

        # Note: final cleanup moved after processing all classes

    # print(f' Distillation complete\n  JVL-C images: {jvl_c_root}')
    # return jvl_c_root

    # Final cleanup: pipelines, models, and loaders (after processing all classes)
    free_memory(
        l2i_pipe,
        jvlc,  # contains VAE + CLIP
        dataloader, path_all, descriptions_map, class_data
    )

    print("Distillation complete")
    print(f"  JVL-C images: {jvl_c_root}")
    return jvl_c_root

# ------------------------------
# 8. Evaluation via Minimax (baseline)
# ------------------------------

def run_minimax_eval(baseline_repo, syn_root, real_root, ipc, seed, repeat, tag, model_name, depth):
    """Invoke the Minimax training script for a given model configuration."""
    eval_dir = os.path.join(baseline_repo, '04_evaluation', 'Minimax')
    train_py = os.path.join(eval_dir, 'train.py')
    cmd = [
        sys.executable, 'train.py',
        '-d', 'imagenet',
        '--imagenet_dir', syn_root, real_root,
        '-n', model_name, '--depth', str(depth),
        '--nclass', '10', '--norm_type', 'instance',
        '--ipc', str(ipc), '--repeat', str(repeat),
        '--spec', 'woof', '--seed', str(seed), '--tag', tag,
        '--slct_type', 'random', '--verbose',
    ]
    subprocess.run(cmd, check=True, cwd=eval_dir)


# Lightweight dependency guard for Minimax
def ensure_minimax_deps():
    """Install lightweight dependencies needed for Minimax evaluation if missing."""
    missing = []
    try:
        import efficientnet_pytorch  # type: ignore
    except Exception:
        missing.append('efficientnet-pytorch')
    try:
        import timm  # type: ignore
    except Exception:
        missing.append('timm')
    if missing:
        print('Installing Minimax deps:', missing)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q'] + missing, check=False)


def posthoc_minimax_eval_from_distilled(
    distilled_seed_root: str,
    baseline_repo: str,
    real_root: str,
    ipc: int,
    seed: int,
    repeat: int,
    tag: str,
    which: str = 'jvl_c',
):
    """Run Minimax using a local distilled seed folder built from JVL-C outputs."""
    ensure_minimax_deps()
    if which.lower() != 'jvl_c':
        raise ValueError("Only 'jvl_c' synthetic outputs are available in this pipeline.")
    syn_root = os.path.join(distilled_seed_root, 'distilled_images_jvl_c')
    if not os.path.isdir(syn_root):
        raise FileNotFoundError(f'Synthetic root not found: {syn_root}')
    if not os.path.isdir(real_root):
        raise FileNotFoundError(f'Real root not found: {real_root}')

    run_minimax_eval(baseline_repo=baseline_repo, syn_root=syn_root, real_root=real_root,
                     ipc=ipc, seed=seed, repeat=repeat, tag=f'{tag}_resnet18', model_name='resnet', depth=18)
    run_minimax_eval(baseline_repo=baseline_repo, syn_root=syn_root, real_root=real_root,
                     ipc=ipc, seed=seed, repeat=repeat, tag=f'{tag}_resnet_ap', model_name='resnet_ap', depth=10)
    run_minimax_eval(baseline_repo=baseline_repo, syn_root=syn_root, real_root=real_root,
                     ipc=ipc, seed=seed, repeat=repeat, tag=f'{tag}_convnet', model_name='convnet', depth=6)


# ------------------------------
# 8a. Compact collapse diagnostics (CLIP-based)
# ------------------------------

def _load_clip_for_diag(device: str):
    """Load CLIP model/processor on the requested device for diagnostics."""
    model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')
    proc = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14')
    return model.to(device).eval(), proc


@torch.no_grad()
def _gather_class_features(root: str, wnid: str, model: CLIPModel, proc: CLIPProcessor, device: str, max_per_class: int = 64) -> Optional[torch.Tensor]:
    """Collect up to max_per_class image features from a class folder using CLIP."""
    import PIL.Image as Image
    cls_dir = os.path.join(root, wnid)
    if not os.path.isdir(cls_dir):
        return None
    imgs = []
    for fn in os.listdir(cls_dir):
        if fn.lower().endswith((".png", ".jpg", ".jpeg")):
            try:
                imgs.append(Image.open(os.path.join(cls_dir, fn)).convert('RGB'))
            except Exception:
                continue
            if len(imgs) >= max_per_class:
                break
    if len(imgs) == 0:
        return None
    inputs = proc(images=imgs, return_tensors='pt', padding=True).to(device)
    feats = model.get_image_features(**inputs)
    feats = feats / feats.norm(dim=-1, keepdim=True)
    return feats


@torch.no_grad()
def _class_text_feature(name: str, model: CLIPModel, proc: CLIPProcessor, device: str) -> torch.Tensor:
    """Compute the normalized CLIP text embedding for a simple class prompt."""
    prompt = f"a photo of a {name}"
    t = proc(text=[prompt], return_tensors='pt', padding=True, truncation=True).to(device)
    tf = model.get_text_features(**t)
    return tf / tf.norm(dim=-1, keepdim=True)


def compute_clip_diagnostics_for_root(root: str, class_ids: List[str], device: str) -> Dict:
    """Compute simple CLIP diagnostics for a synthetic root with WNID subfolders."""
    model, proc = _load_clip_for_diag(device)
    per_class = {}
    centroids = {}
    align_scores = []

    for wnid in class_ids:
        feats = _gather_class_features(root, wnid, model, proc, device)
        if feats is None:
            continue
        mu = feats.mean(dim=0, keepdim=True)
        mu = mu / mu.norm(dim=-1, keepdim=True)
        # intra dispersion: mean(1 - cos(feat, mu))
        intra = (1.0 - (feats @ mu.t()).squeeze(-1)).mean().item()
        per_class[wnid] = {
            'count': feats.shape[0],
            'intra_dispersion': intra,
        }
        centroids[wnid] = mu.squeeze(0)

        # alignment vs class name text feature
        name = IMAGEWOOF_CLASS_MAPPING.get(wnid, wnid)
        tf = _class_text_feature(name, model, proc, device)
        align = (mu @ tf.t()).item()
        align_scores.append(align)

    # inter-class separation among centroids
    wnids_present = list(centroids.keys())
    inter_vals = []
    for i in range(len(wnids_present)):
        for j in range(i + 1, len(wnids_present)):
            ci = centroids[wnids_present[i]]
            cj = centroids[wnids_present[j]]
            inter_vals.append(1.0 - torch.dot(ci, cj).item())

    avg_intra = float(np.mean([v['intra_dispersion'] for v in per_class.values()])) if per_class else float('nan')
    inter_sep = float(np.mean(inter_vals)) if inter_vals else float('nan')
    align_mean = float(np.mean(align_scores)) if align_scores else float('nan')

    return {
        'per_class': per_class,
        'avg_intra': avg_intra,
        'inter_sep': inter_sep,
        'align': align_mean,
        'classes_covered': len(per_class),
    }

# ------------------------------
# 8b. Feedback loop to tune JVL-C
# ------------------------------

def _read_class_ids(class_file: str) -> List[str]:
    """Read non-empty class IDs (one per line) from a text file."""
    return [ln.strip() for ln in open(class_file, 'r') if ln.strip()]


def resolve_class_file_for_diag(cfg: Config, baseline_repo: str) -> str:
    """Resolve a valid class_woof.txt for diagnostics.

    Preference order:
    1) cfg.class_file if present
    2) baseline repo label-prompt (both spellings)
    3) Minimax misc/class_woof.txt
    4) Create cfg.class_file from IMAGENETTE mapping and return it
    """
    candidates = [
        cfg.class_file,
        os.path.join(baseline_repo, '03_distiilation', 'label-prompt', 'imagenet_woof_classes.txt'),
        os.path.join(baseline_repo, '03_distillation', 'label-prompt', 'imagenet_woof_classes.txt'),
        os.path.join(baseline_repo, '04_evaluation', 'Minimax', 'misc', 'class_woof.txt'),
    ]

    for p in candidates:
        if p and os.path.isfile(p):
            return p
    # Fallback: create cfg.class_file from known wnids
    ensure_class_file(cfg.class_file)
    return cfg.class_file


# ------------------------------
# 9. Main entry utility (leave steps callable individually)
# ------------------------------

def assert_metadata_present(config: Config):
    """Raise if metadata.jsonl is missing; keeps the notebook fail-fast."""
    if not os.path.isfile(config.metadata_file):
        raise FileNotFoundError(
            f"metadata.jsonl not found at {config.metadata_file}.\n"
            "Please run baselinesetup.py's dataset/metadata step to create it."
        )


def sanity_check_imagewoof(config: Config, baseline_repo: str):
    """Sanity checks for class mapping, directories, and metadata consistency.
    Raises AssertionError/FileNotFoundError on problems to surface issues in one notebook.
    """
    # 1) Resolve class list source
    baseline_class_file = os.path.join(baseline_repo, '03_distiilation', 'label-prompt', 'class_woof.txt')
    class_file_path = config.class_file if os.path.isfile(config.class_file) else (
        baseline_class_file if os.path.isfile(baseline_class_file) else None
    )
    if class_file_path is None:
        raise FileNotFoundError(
            f"class_woof.txt not found at {config.class_file} nor {baseline_class_file}."
        )
    with open(class_file_path, 'r') as f:
        wnids = [ln.strip() for ln in f if ln.strip()]
    assert len(wnids) == 10, f"Expected 10 WNIDs, got {len(wnids)} from {class_file_path}"

    # 2) Check mapping keys match wnids
    map_keys = set(IMAGEWOOF_CLASS_MAPPING.keys())
    set_wnids = set(wnids)
    assert map_keys == set_wnids, (
        f"IMAGEWOOF_CLASS_MAPPING keys mismatch.\n"
        f"Mapping-only: {sorted(map_keys - set_wnids)}\n"
        f"Classfile-only: {sorted(set_wnids - map_keys)}"
    )

    # 3) Check train/val directories exist and contain wnid subfolders
    assert os.path.isdir(config.train_dir), f"Missing train dir: {config.train_dir}"
    assert os.path.isdir(config.val_dir), f"Missing val dir: {config.val_dir}"
    train_subs = sorted([d for d in os.listdir(config.train_dir) if os.path.isdir(os.path.join(config.train_dir, d))])
    val_subs = sorted([d for d in os.listdir(config.val_dir) if os.path.isdir(os.path.join(config.val_dir, d))])
    assert set(train_subs) == set_wnids, (
        f"Train subfolders mismatch class file.\n"
        f"Extra in train: {sorted(set(train_subs) - set_wnids)}\n"
        f"Missing in train: {sorted(set_wnids - set(train_subs))}"
    )
    # Val may be missing in some workflows; if present, ensure subset match
    if len(val_subs) > 0:
        assert set(val_subs) == set_wnids, (
            f"Val subfolders mismatch class file.\n"
            f"Extra in val: {sorted(set(val_subs) - set_wnids)}\n"
            f"Missing in val: {sorted(set_wnids - set(val_subs))}"
        )

    # 4) Verify ImageFolder classes vs class file
    ds = datasets.ImageFolder(root=config.train_dir)
    ds_classes = set(ds.classes)
    assert ds_classes == set_wnids, (
        f"ImageFolder classes mismatch class file.\n"
        f"Extra in ImageFolder: {sorted(ds_classes - set_wnids)}\n"
        f"Missing in ImageFolder: {sorted(set_wnids - ds_classes)}"
    )

    # 5) Verify metadata.jsonl paths exist
    assert os.path.isfile(config.metadata_file), f"Missing metadata.jsonl: {config.metadata_file}"
    checked, missing = 0, 0
    with open(config.metadata_file, 'r', encoding='utf-8') as f:
        for line in f:
            if checked >= 200:
                break
            if not line.strip():
                continue
            rec = json.loads(line)
            rel = rec.get('file_name', '')
            if not rel:
                missing += 1
                checked += 1
                continue
            p1 = os.path.join(config.train_dir, rel)
            # Some datasets include _WNID suffix in filenames; try alternate
            wnid = rel.split('/')[0]
            base = os.path.basename(rel)
            alt = os.path.join(config.train_dir, wnid, base.replace(f'_{wnid}.JPEG', '.JPEG'))
            if not (os.path.isfile(p1) or os.path.isfile(alt)):
                missing += 1
            checked += 1
    assert missing == 0, f"metadata.jsonl has {missing}/{checked} missing files under {config.train_dir}"

    print("Class/map/metadata sanity checks passed")


def main(seed: int = 0, run_eval: bool = True, finetuned_model_override: Optional[str] = None):
    """End-to-end runner: set up paths, select α, distill JVL-C prototypes, and optionally evaluate."""
    try_mount_drive()
    cfg = Config(base_seed=seed)
    cfg.set_seed(seed)
    os.makedirs(cfg.outputs_root, exist_ok=True)
    os.makedirs(cfg.distilled_root, exist_ok=True)
    os.makedirs(cfg.prototype_root, exist_ok=True)

    baseline_repo = resolve_baseline_repo(project_root=os.getcwd())
    print(f'Baseline repo: {baseline_repo}')

    # Do not rebuild metadata here; must be prepared by baselinesetup.py
    assert_metadata_present(cfg)
    sanity_check_imagewoof(cfg, baseline_repo)


    # Principled Fisher-on-sphere global α selection
    best_alpha = select_alpha_fisher_on_sphere(
        cfg=cfg,
        baseline_repo=baseline_repo,
        a_min=cfg.jvlc_global_alpha_min,
        a_max=cfg.jvlc_global_alpha_max,
        weights=(0.25, 0.50, 0.25),
        device=cfg.device,
    )
    print(f"[Run] α* (Fisher-on-sphere) selected: {best_alpha:.3f}")
    cfg.alpha = best_alpha

    # Run a single-pass JVL-C with the chosen Fisher α (feedback tuning disabled)
    print(f"\n[Run] JVL-C distillation (global α={cfg.alpha:.2f})")
    jvl_c_root = run_jvl_distillation(
        config=cfg,
        baseline_repo=baseline_repo,
        finetuned_model_path=finetuned_model_override,
    )


    # Optional: clear residual caches
    import gc, torch
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

    # Evaluation (Minimax)
    if run_eval:
        # CLIP diagnostics snapshot for the final JVL-C outputs
        class_file_path = resolve_class_file_for_diag(cfg, baseline_repo)
        class_ids = [ln.strip() for ln in open(class_file_path, 'r') if ln.strip()]
        diag_jvlc = compute_clip_diagnostics_for_root(
            os.path.join(cfg.distilled_root, 'distilled_images_jvl_c'), class_ids, cfg.device
        )
        print(f"Final JVL-C diag -> intra={diag_jvlc.get('avg_intra'):.4f} | inter_sep={diag_jvlc.get('inter_sep'):.4f} | align={diag_jvlc.get('align'):.4f}")

        # Post-hoc evaluation directly from local distilled folder
        try:
            posthoc_minimax_eval_from_distilled(
                distilled_seed_root=cfg.distilled_root,
                baseline_repo=baseline_repo,
                real_root=cfg.data_root,
                ipc=cfg.ipc,
                seed=seed,
                repeat=cfg.eval_repeat,
                tag=f'JVL-C_seed{seed}',
                which='jvl_c',
            )
        except subprocess.CalledProcessError as e:
            print('Minimax evaluation failed with exit code:', e.returncode)
            print('Retrying after ensuring dependencies...')
            ensure_minimax_deps()
            posthoc_minimax_eval_from_distilled(
                distilled_seed_root=cfg.distilled_root,
                baseline_repo=baseline_repo,
                real_root=cfg.data_root,
                ipc=cfg.ipc,
                seed=seed,
                repeat=cfg.eval_repeat,
                tag=f'JVL-C_seed{seed}',
                which='jvl_c',
            )

if __name__ == '__main__':
    main(seed=0, run_eval=True)